In [2]:
# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex
import fastparquet # fast read/write for large data structures
import sklearn.preprocessing as pre # for data normalisation
from sklearn.metrics import pairwise_distances

import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package
import SPOTSAR_main as sm


In [ ]:
print(cm.vik)

In [3]:
################ Define user INPUTS #######################
######## please edit the values of this block only ########
###########################################################

# define hillshade file
HS_FILE = './test_data/DEM/TDX_Merapi_WGS84_HS.tif'

# define lon and lat files
LON_FILE = './test_data/CSK_dsc/geo2/20200910.lon'
LAT_FILE = './test_data/CSK_dsc/geo2/20200910.lat'

# define parameter text file
PARAM_FILE = './test_data/CSK_dsc/params.txt'

# define map region of interest
lon_lims = [110.425, 110.45]
lat_lims = [-7.555, -7.535]

# define colour range {min max} (min = -max)
vmax = 3 # range of colourscale in meters

# define file names for data, lon and lat
DIRECTORY_PATH = "./test_data/CSK_dsc/DISP_txt4/"
# define path to ccp and ccs files
DIRECTORY_PATH_CCS = "./test_data/CSK_dsc/CCS4/"

# Set the regular expression pattern to match the file names
# PATTERN1 = r"^c20200927_c20201113_disp_[0-9]+_[0-9]+\.txt$"
# PATTERN2 = r"^c20200926_c20201113_disp_[0-9]+_[0-9]+\.txt$"
# PATTERN3 = r"^c20200927_c20210812_disp_[0-9]+_[0-9]+\.txt$"
# PATTERN4 = r"^c20210217_c20210218_disp_[0-9]+_[0-9]+\.txt$"

PATTERN1 = r"^c20210406_c20210508_disp_[0-9]+_[0-9]+\.txt$"

# Set the regular expression pattern to match the ccs file names
# PATTERN_CCS1 = r"^c20200927_c20201113_ccs_[0-9]+_[0-9]+$"
# PATTERN_CCS2 = r"^c20200926_c20201113_ccs_[0-9]+_[0-9]+$"
# PATTERN_CCS3 = r"^c20200927_c20210812_ccs_[0-9]+_[0-9]+$"
# PATTERN_CCS4 = r"^c20210217_c20210218_ccs_[0-9]+_[0-9]+$"

PATTERN_CCS1 = r"^c20210406_c20210508_ccs_[0-9]+_[0-9]+$"



# open hillshade file and re-order offset and CCS files

# open hill shade file with rasterio
DEM_HS = rio.open(HS_FILE)
SHADING = DEM_HS.read(1,masked=True) # rasterio bands are indexed from 1

# extract DEM extent
DEM_EXTENT=[DEM_HS.bounds.left,DEM_HS.bounds.right,DEM_HS.bounds.bottom,DEM_HS.bounds.top]

# read parameters from text file
config = configparser.ConfigParser()
config.read(PARAM_FILE)
WIDTH = int(config.get('params', 'width'))
LINES = int(config.get('params', 'lines'))
WIDTH_CCS = int(config.get('params', 'width_ccs'))
LINES_CCS = int(config.get('params', 'lines_ccs'))
R_START = int(config.get('params', 'r_start'))
A_START = int(config.get('params', 'a_start'))
R_STEP = int(config.get('params', 'r_step'))
A_STEP = int(config.get('params', 'a_step'))
HEADING = float(config.get('params', 'heading'))
MEAN_INC = float(config.get('params', 'mean_inc'))

In [5]:

# reorder file using Post_processing.reorder_files
matching_files1 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN1,0)
matching_files_ccs1 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS1,0)
# matching_files2 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN2,0)
# matching_files_ccs2 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS2,0)
# matching_files3 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN3,0)
# matching_files_ccs3 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS3,0)
# matching_files4 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN4,0)
# matching_files_ccs4 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS4,0)


# test if file ordering has worked
print(matching_files1)
print(matching_files_ccs1)
# print(matching_files2)
# print(matching_files_ccs2)
# print(matching_files3)
# print(matching_files_ccs3)
# print(matching_files4)
# print(matching_files_ccs4)

['c20210406_c20210508_disp_74_38.txt', 'c20210406_c20210508_disp_148_72.txt', 'c20210406_c20210508_disp_224_108.txt']
['c20210406_c20210508_ccs_74_38', 'c20210406_c20210508_ccs_148_72', 'c20210406_c20210508_ccs_224_108']


In [6]:

# load data from files into class multi-kernel
example_pairs = []
# for (matching_files,matching_files_ccs) in zip([matching_files1,matching_files2,matching_files3,matching_files4],[matching_files_ccs1,matching_files_ccs2,matching_files_ccs3,matching_files_ccs4]):
#     datastack = sm.Post_processing.MultiKernel(DIRECTORY_PATH,
#                                             matching_files,
#                                             DIRECTORY_PATH_CCS,
#                                             matching_files_ccs,
#                                             LAT_FILE,
#                                             LON_FILE,
#                                             HEADING,
#                                             MEAN_INC,
#                                             LINES_CCS,
#                                             WIDTH_CCS)
#     # We need to assign some data not stored in the disp.txt files.
#     datastack.get_params_from_file_name()
#     datastack.get_latlon_from_file(WIDTH)
#     datastack.add_lat_lon_to_data(R_START,A_START)
#     datastack.crop_stack_ccs(R_STEP,A_STEP)
#     # the object datastack now has several attributes associated with the whole dataset (e.g., date1, date2, heading)
#     # Next we add all the offset data (disp.txt) to the stack
#     stacked_data = datastack.assign_data_to_stack(R_STEP,A_STEP)
#     # The attribute 'Stack' we find a list of single-kernel objects which contain the actual offset data, ccp and ccs data and the coordinates.

#     # add stack to list
#     example_pairs.append(datastack)

# for (matching_files,matching_files_ccs) in zip([matching_files1,matching_files2,matching_files3,matching_files4],[matching_files_ccs1,matching_files_ccs2,matching_files_ccs3,matching_files_ccs4]):
datastack = sm.Post_processing.MultiKernel(DIRECTORY_PATH,
                                        matching_files1,
                                        DIRECTORY_PATH_CCS,
                                        matching_files_ccs1,
                                        LAT_FILE,
                                        LON_FILE,
                                        HEADING,
                                        MEAN_INC,
                                        LINES_CCS,
                                        WIDTH_CCS)
# We need to assign some data not stored in the disp.txt files.
datastack.get_params_from_file_name()
datastack.get_latlon_from_file(WIDTH)
datastack.add_lat_lon_to_data(R_START,A_START)
datastack.crop_stack_ccs(R_STEP,A_STEP)
# the object datastack now has several attributes associated with the whole dataset (e.g., date1, date2, heading)
# Next we add all the offset data (disp.txt) to the stack
stacked_data = datastack.assign_data_to_stack(R_STEP,A_STEP)
# The attribute 'Stack' we find a list of single-kernel objects which contain the actual offset data, ccp and ccs data and the coordinates.

# add stack to list
example_pairs.append(datastack)

In [ ]:

# save_keys = ['R_idx','A_idx','Row_index','Col_index','Lon_off','Lat_off','R_off','A_off']
# for key in dir(obj[0]):
#     if ('HDBSCAN' == key[0:7]) and (key[-3:] != 'vec'):
#         save_keys.append(key)
#     if ('GLOSH' == key[0:5]) and  (key[-3:] != 'vec'):
#         save_keys.append(key)
#     if ('LOF' == key[0:3]) and  (key[-3:] != 'vec'):
#         save_keys.append(key)
# print(save_keys)
# obj[0].to_hdf5(f'./test_data/CSK_dsc/hdf5_files/{obj[0].Name[0:-4]}_outlier_detected.h5',save_keys)
objs = example_pairs[0].Stack

files = [f'./test_data/CSK_dsc/hdf5_files/{example_pairs[0].Stack[0].Name[0:-4]}_outlier_detected2.h5',
         f'./test_data/CSK_dsc/hdf5_files/{example_pairs[0].Stack[1].Name[0:-4]}_outlier_detected_rot.h5',
         f'./test_data/CSK_dsc/hdf5_files/{example_pairs[0].Stack[2].Name[0:-4]}_outlier_detected_rot.h5',
         f'./test_data/CSK_dsc/hdf5_files/{example_pairs[0].Stack[3].Name[0:-4]}_outlier_detected_rot.h5',
         f'./test_data/CSK_dsc/hdf5_files/{example_pairs[0].Stack[4].Name[0:-4]}_outlier_detected_rot.h5',
         ]
# f = h5py.File(f'./test_data/CSK_dsc/hdf5_files/{obj.Name[0:-4]}_outlier_detected.h5')
query_keys = ['HDBSCAN_labels_100_1', 'HDBSCAN_labels_100_10', 'HDBSCAN_labels_100_10_vec', 'HDBSCAN_labels_100_1_vec', 'HDBSCAN_labels_100_20', 'HDBSCAN_labels_100_20_vec', 'HDBSCAN_labels_100_30', 'HDBSCAN_labels_100_30_vec', 'HDBSCAN_labels_100_40', 'HDBSCAN_labels_100_40_vec', 'HDBSCAN_labels_100_50', 'HDBSCAN_labels_100_50_vec', 'HDBSCAN_labels_150_1', 'HDBSCAN_labels_150_15', 'HDBSCAN_labels_150_15_vec', 'HDBSCAN_labels_150_1_vec', 'HDBSCAN_labels_150_30', 'HDBSCAN_labels_150_30_vec', 'HDBSCAN_labels_150_45', 'HDBSCAN_labels_150_45_vec', 'HDBSCAN_labels_150_60', 'HDBSCAN_labels_150_60_vec', 'HDBSCAN_labels_150_75', 'HDBSCAN_labels_150_75_vec', 'HDBSCAN_labels_200_1', 'HDBSCAN_labels_200_100', 'HDBSCAN_labels_200_100_vec', 'HDBSCAN_labels_200_1_vec', 'HDBSCAN_labels_200_20', 'HDBSCAN_labels_200_20_vec', 'HDBSCAN_labels_200_40', 'HDBSCAN_labels_200_40_vec', 'HDBSCAN_labels_200_60', 'HDBSCAN_labels_200_60_vec', 'HDBSCAN_labels_200_80', 'HDBSCAN_labels_200_80_vec', 'HDBSCAN_labels_250_1', 'HDBSCAN_labels_250_100', 'HDBSCAN_labels_250_100_vec', 'HDBSCAN_labels_250_125', 'HDBSCAN_labels_250_125_vec', 'HDBSCAN_labels_250_1_vec', 'HDBSCAN_labels_250_25', 'HDBSCAN_labels_250_25_vec', 'HDBSCAN_labels_250_50', 'HDBSCAN_labels_250_50_vec', 'HDBSCAN_labels_250_75', 'HDBSCAN_labels_250_75_vec', 'HDBSCAN_labels_300_1', 'HDBSCAN_labels_300_120', 'HDBSCAN_labels_300_120_vec', 'HDBSCAN_labels_300_150', 'HDBSCAN_labels_300_150_vec', 'HDBSCAN_labels_300_1_vec', 'HDBSCAN_labels_300_30', 'HDBSCAN_labels_300_30_vec', 'HDBSCAN_labels_300_60', 'HDBSCAN_labels_300_60_vec', 'HDBSCAN_labels_300_90', 'HDBSCAN_labels_300_90_vec', 'HDBSCAN_labels_400_1', 'HDBSCAN_labels_400_120', 'HDBSCAN_labels_400_120_vec', 'HDBSCAN_labels_400_160', 'HDBSCAN_labels_400_160_vec', 'HDBSCAN_labels_400_1_vec', 'HDBSCAN_labels_400_200', 'HDBSCAN_labels_400_200_vec', 'HDBSCAN_labels_400_40', 'HDBSCAN_labels_400_40_vec', 'HDBSCAN_labels_400_80', 'HDBSCAN_labels_400_80_vec', 'HDBSCAN_labels_500_1', 'HDBSCAN_labels_500_100', 'HDBSCAN_labels_500_100_vec', 'HDBSCAN_labels_500_150', 'HDBSCAN_labels_500_150_vec', 'HDBSCAN_labels_500_1_vec', 'HDBSCAN_labels_500_200', 'HDBSCAN_labels_500_200_vec', 'HDBSCAN_labels_500_250', 'HDBSCAN_labels_500_250_vec', 'HDBSCAN_labels_500_50', 'HDBSCAN_labels_500_50_vec', 'HDBSCAN_labels_50_1', 'HDBSCAN_labels_50_10', 'HDBSCAN_labels_50_10_vec', 'HDBSCAN_labels_50_15', 'HDBSCAN_labels_50_15_vec', 'HDBSCAN_labels_50_1_vec', 'HDBSCAN_labels_50_20', 'HDBSCAN_labels_50_20_vec', 'HDBSCAN_labels_50_25', 'HDBSCAN_labels_50_25_vec', 'HDBSCAN_labels_50_5', 'HDBSCAN_labels_50_5_vec', 'HDBSCAN_outlier_scores_100_1', 'HDBSCAN_outlier_scores_100_10', 'HDBSCAN_outlier_scores_100_10_vec', 'HDBSCAN_outlier_scores_100_1_vec', 'HDBSCAN_outlier_scores_100_20', 'HDBSCAN_outlier_scores_100_20_vec', 'HDBSCAN_outlier_scores_100_30', 'HDBSCAN_outlier_scores_100_30_vec', 'HDBSCAN_outlier_scores_100_40', 'HDBSCAN_outlier_scores_100_40_vec', 'HDBSCAN_outlier_scores_100_50', 'HDBSCAN_outlier_scores_100_50_vec', 'HDBSCAN_outlier_scores_150_1', 'HDBSCAN_outlier_scores_150_15', 'HDBSCAN_outlier_scores_150_15_vec', 'HDBSCAN_outlier_scores_150_1_vec', 'HDBSCAN_outlier_scores_150_30', 'HDBSCAN_outlier_scores_150_30_vec', 'HDBSCAN_outlier_scores_150_45', 'HDBSCAN_outlier_scores_150_45_vec', 'HDBSCAN_outlier_scores_150_60', 'HDBSCAN_outlier_scores_150_60_vec', 'HDBSCAN_outlier_scores_150_75', 'HDBSCAN_outlier_scores_150_75_vec', 'HDBSCAN_outlier_scores_200_1', 'HDBSCAN_outlier_scores_200_100', 'HDBSCAN_outlier_scores_200_100_vec', 'HDBSCAN_outlier_scores_200_1_vec', 'HDBSCAN_outlier_scores_200_20', 'HDBSCAN_outlier_scores_200_20_vec', 'HDBSCAN_outlier_scores_200_40', 'HDBSCAN_outlier_scores_200_40_vec', 'HDBSCAN_outlier_scores_200_60', 'HDBSCAN_outlier_scores_200_60_vec', 'HDBSCAN_outlier_scores_200_80', 'HDBSCAN_outlier_scores_200_80_vec', 'HDBSCAN_outlier_scores_250_1', 'HDBSCAN_outlier_scores_250_100', 'HDBSCAN_outlier_scores_250_100_vec', 'HDBSCAN_outlier_scores_250_125', 'HDBSCAN_outlier_scores_250_125_vec', 'HDBSCAN_outlier_scores_250_1_vec', 'HDBSCAN_outlier_scores_250_25', 'HDBSCAN_outlier_scores_250_25_vec', 'HDBSCAN_outlier_scores_250_50', 'HDBSCAN_outlier_scores_250_50_vec', 'HDBSCAN_outlier_scores_250_75', 'HDBSCAN_outlier_scores_250_75_vec', 'HDBSCAN_outlier_scores_300_1', 'HDBSCAN_outlier_scores_300_120', 'HDBSCAN_outlier_scores_300_120_vec', 'HDBSCAN_outlier_scores_300_150', 'HDBSCAN_outlier_scores_300_150_vec', 'HDBSCAN_outlier_scores_300_1_vec', 'HDBSCAN_outlier_scores_300_30', 'HDBSCAN_outlier_scores_300_30_vec', 'HDBSCAN_outlier_scores_300_60', 'HDBSCAN_outlier_scores_300_60_vec', 'HDBSCAN_outlier_scores_300_90', 'HDBSCAN_outlier_scores_300_90_vec', 'HDBSCAN_outlier_scores_400_1', 'HDBSCAN_outlier_scores_400_120', 'HDBSCAN_outlier_scores_400_120_vec', 'HDBSCAN_outlier_scores_400_160', 'HDBSCAN_outlier_scores_400_160_vec', 'HDBSCAN_outlier_scores_400_1_vec', 'HDBSCAN_outlier_scores_400_200', 'HDBSCAN_outlier_scores_400_200_vec', 'HDBSCAN_outlier_scores_400_40', 'HDBSCAN_outlier_scores_400_40_vec', 'HDBSCAN_outlier_scores_400_80', 'HDBSCAN_outlier_scores_400_80_vec', 'HDBSCAN_outlier_scores_500_1', 'HDBSCAN_outlier_scores_500_100', 'HDBSCAN_outlier_scores_500_100_vec', 'HDBSCAN_outlier_scores_500_150', 'HDBSCAN_outlier_scores_500_150_vec', 'HDBSCAN_outlier_scores_500_1_vec', 'HDBSCAN_outlier_scores_500_200', 'HDBSCAN_outlier_scores_500_200_vec', 'HDBSCAN_outlier_scores_500_250', 'HDBSCAN_outlier_scores_500_250_vec', 'HDBSCAN_outlier_scores_500_50', 'HDBSCAN_outlier_scores_500_50_vec', 'HDBSCAN_outlier_scores_50_1', 'HDBSCAN_outlier_scores_50_10', 'HDBSCAN_outlier_scores_50_10_vec', 'HDBSCAN_outlier_scores_50_15', 'HDBSCAN_outlier_scores_50_15_vec', 'HDBSCAN_outlier_scores_50_1_vec', 'HDBSCAN_outlier_scores_50_20', 'HDBSCAN_outlier_scores_50_20_vec', 'HDBSCAN_outlier_scores_50_25', 'HDBSCAN_outlier_scores_50_25_vec', 'HDBSCAN_outlier_scores_50_5', 'HDBSCAN_outlier_scores_50_5_vec', 'HDBSCAN_probabilities_100_1', 'HDBSCAN_probabilities_100_10', 'HDBSCAN_probabilities_100_10_vec', 'HDBSCAN_probabilities_100_1_vec', 'HDBSCAN_probabilities_100_20', 'HDBSCAN_probabilities_100_20_vec', 'HDBSCAN_probabilities_100_30', 'HDBSCAN_probabilities_100_30_vec', 'HDBSCAN_probabilities_100_40', 'HDBSCAN_probabilities_100_40_vec', 'HDBSCAN_probabilities_100_50', 'HDBSCAN_probabilities_100_50_vec', 'HDBSCAN_probabilities_150_1', 'HDBSCAN_probabilities_150_15', 'HDBSCAN_probabilities_150_15_vec', 'HDBSCAN_probabilities_150_1_vec', 'HDBSCAN_probabilities_150_30', 'HDBSCAN_probabilities_150_30_vec', 'HDBSCAN_probabilities_150_45', 'HDBSCAN_probabilities_150_45_vec', 'HDBSCAN_probabilities_150_60', 'HDBSCAN_probabilities_150_60_vec', 'HDBSCAN_probabilities_150_75', 'HDBSCAN_probabilities_150_75_vec', 'HDBSCAN_probabilities_200_1', 'HDBSCAN_probabilities_200_100', 'HDBSCAN_probabilities_200_100_vec', 'HDBSCAN_probabilities_200_1_vec', 'HDBSCAN_probabilities_200_20', 'HDBSCAN_probabilities_200_20_vec', 'HDBSCAN_probabilities_200_40', 'HDBSCAN_probabilities_200_40_vec', 'HDBSCAN_probabilities_200_60', 'HDBSCAN_probabilities_200_60_vec', 'HDBSCAN_probabilities_200_80', 'HDBSCAN_probabilities_200_80_vec', 'HDBSCAN_probabilities_250_1', 'HDBSCAN_probabilities_250_100', 'HDBSCAN_probabilities_250_100_vec', 'HDBSCAN_probabilities_250_125', 'HDBSCAN_probabilities_250_125_vec', 'HDBSCAN_probabilities_250_1_vec', 'HDBSCAN_probabilities_250_25', 'HDBSCAN_probabilities_250_25_vec', 'HDBSCAN_probabilities_250_50', 'HDBSCAN_probabilities_250_50_vec', 'HDBSCAN_probabilities_250_75', 'HDBSCAN_probabilities_250_75_vec', 'HDBSCAN_probabilities_300_1', 'HDBSCAN_probabilities_300_120', 'HDBSCAN_probabilities_300_120_vec', 'HDBSCAN_probabilities_300_150', 'HDBSCAN_probabilities_300_150_vec', 'HDBSCAN_probabilities_300_1_vec', 'HDBSCAN_probabilities_300_30', 'HDBSCAN_probabilities_300_30_vec', 'HDBSCAN_probabilities_300_60', 'HDBSCAN_probabilities_300_60_vec', 'HDBSCAN_probabilities_300_90', 'HDBSCAN_probabilities_300_90_vec', 'HDBSCAN_probabilities_400_1', 'HDBSCAN_probabilities_400_120', 'HDBSCAN_probabilities_400_120_vec', 'HDBSCAN_probabilities_400_160', 'HDBSCAN_probabilities_400_160_vec', 'HDBSCAN_probabilities_400_1_vec', 'HDBSCAN_probabilities_400_200', 'HDBSCAN_probabilities_400_200_vec', 'HDBSCAN_probabilities_400_40', 'HDBSCAN_probabilities_400_40_vec', 'HDBSCAN_probabilities_400_80', 'HDBSCAN_probabilities_400_80_vec', 'HDBSCAN_probabilities_500_1', 'HDBSCAN_probabilities_500_100', 'HDBSCAN_probabilities_500_100_vec', 'HDBSCAN_probabilities_500_150', 'HDBSCAN_probabilities_500_150_vec', 'HDBSCAN_probabilities_500_1_vec', 'HDBSCAN_probabilities_500_200', 'HDBSCAN_probabilities_500_200_vec', 'HDBSCAN_probabilities_500_250', 'HDBSCAN_probabilities_500_250_vec', 'HDBSCAN_probabilities_500_50', 'HDBSCAN_probabilities_500_50_vec', 'HDBSCAN_probabilities_50_1', 'HDBSCAN_probabilities_50_10', 'HDBSCAN_probabilities_50_10_vec', 'HDBSCAN_probabilities_50_15', 'HDBSCAN_probabilities_50_15_vec', 'HDBSCAN_probabilities_50_1_vec', 'HDBSCAN_probabilities_50_20', 'HDBSCAN_probabilities_50_20_vec', 'HDBSCAN_probabilities_50_25', 'HDBSCAN_probabilities_50_25_vec', 'HDBSCAN_probabilities_50_5', 'HDBSCAN_probabilities_50_5_vec', 'LOF_labels_100', 'LOF_labels_100_vec', 'LOF_labels_150', 'LOF_labels_150_vec', 'LOF_labels_200', 'LOF_labels_200_vec', 'LOF_labels_250', 'LOF_labels_250_vec', 'LOF_labels_300', 'LOF_labels_300_vec', 'LOF_labels_400', 'LOF_labels_400_vec', 'LOF_labels_50', 'LOF_labels_500', 'LOF_labels_500_vec', 'LOF_labels_50_vec', 'LOF_outlier_score_100_vec', 'LOF_outlier_score_150_vec', 'LOF_outlier_score_200_vec', 'LOF_outlier_score_250_vec', 'LOF_outlier_score_300_vec', 'LOF_outlier_score_400_vec', 'LOF_outlier_score_500_vec', 'LOF_outlier_score_50_vec', 'LOF_outlier_scores_100', 'LOF_outlier_scores_150', 'LOF_outlier_scores_200', 'LOF_outlier_scores_250', 'LOF_outlier_scores_300', 'LOF_outlier_scores_400', 'LOF_outlier_scores_50', 'LOF_outlier_scores_500', 'Mag_off_med_diff_2', 'Mag_off_med_diff_2_vec', 'Mag_off_med_diff_3', 'Mag_off_med_diff_3_vec', 'Mag_off_med_diff_4', 'Mag_off_med_diff_4_vec', 'Mag_off_med_diff_5', 'Mag_off_med_diff_5_vec', 'Mag_off_med_diff_6', 'Mag_off_med_diff_6_vec', 'Mag_off_med_diff_7', 'Mag_off_med_diff_7_vec', 'Mag_off_med_diff_8', 'Mag_off_med_diff_8_vec', 'Mag_off_med_diff_9', 'Mag_off_med_diff_9_vec']

for (obj,file) in zip(objs,files):
    print(obj,file)
    obj.from_hdf5(file,query_keys)

In [7]:
print(dir(example_pairs[0].Stack[0]))

['A_idx', 'A_idx_vec', 'A_off', 'A_off_vec', 'A_win', 'Ccp_off', 'Ccp_off_vec', 'Ccs_off', 'Ccs_off_vec', 'Col_index', 'Col_index_vec', 'Date1', 'Date2', 'Heading', 'Lat_off', 'Lat_off_vec', 'Lon_off', 'Lon_off_vec', 'Mag', 'Mag_vec', 'Name', 'Nan_mask', 'Nan_mask_vec', 'Phase', 'Phase_vec', 'R_idx', 'R_idx_vec', 'R_off', 'R_off_vec', 'R_win', 'Row_index', 'Row_index_vec', 'SNR', 'SNR_vec', 'X_off', 'X_off_vec', 'Y_off', 'Y_off_vec', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'calc_Mag', 'calc_SNR', 'calc_local_L2', 'calc_phase', 'comp_ll_dist_matrix', 'from_hdf5', 'get_Row_Col_vec', 'get_Row_col_idx', 'get_attr_list', 'get_coords', 'get_data', 'get_data_4_wL2', 'get_dates', 'get_vec_data',

In [8]:
import matplotlib.patches as mpatches
import matplotlib.transforms as mtransforms

def add_right_cax(ax, pad, width):
    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos.x1 + pad,
        axpos.y0,
        axpos.x1 + pad + width,
        axpos.y1
    )
    cax = ax.figure.add_axes(caxpos)

    return cax

In [ ]:
%matplotlib osx



In [ ]:
%matplotlib osx
import pickle
import itertools
import random

from sklearn import metrics
from matplotlib.collections import LineCollection
plt.close('all')

# plotting conditions
plotting = 0
plotting_flag = 0
labels = 0
verbose= False

# dataset to test
obj = example_pairs[0].Stack[1]

min_samples_facts = [0.00001, 0.1, 0.2, 0.3, 0.4, 0.5]
min_cluster_sizes = [50,100,150,200,250,300,400,500]
filter_radius = [2,3,4,5,6,7,8,9]

n_cat = 50 # number of samples from each catergory of inlier/outlier
n_runs = 1 # number of runs

# get data
r_win = getattr(obj, "R_win")
a_win = getattr(obj, "A_win")
r_idx = getattr(obj, "R_idx")
a_idx = getattr(obj, "A_idx")

# get testing data
test_data1 = pd.read_csv(
            f"./test_data/CSK_dsc/{obj.Name[:-4]}_test_set.csv", header=0
        ).to_numpy()
test_data2 = pd.read_csv(
            f"./test_data/CSK_dsc/{obj.Name[:-4]}_test_set3.csv", header=0
        ).to_numpy()

test_data3 = pd.read_csv(
            f"./test_data/CSK_dsc/{obj.Name[:-4]}_test_set4.csv", header=0
        ).to_numpy()

# select which test data to use
test_data4 = np.row_stack((test_data1,test_data2,test_data3))

test_data4 = np.unique(test_data4,axis=0)


In [ ]:
# seperate based on category
# n_cat = 60
# test_data3 = test_data3[test_data3[:,3]==0]
sel0 = test_data4[test_data4[:,2]==0]
sel1 = test_data4[test_data4[:,2]==1]
sel2 = test_data4[test_data4[:,2]==2]
sel3 = test_data4[test_data4[:,2]==3]

# initialize outlut lists
auc_list = []
fpr_list = []
tpr_list = []
thresh_list = []

auc_mag_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
auc_med_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
auc_hdb_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
auc_glsh_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
auc_lof_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
auc_comb123_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
auc_comb23_mat = np.empty((np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs))
# main loop
for run_id in range(n_runs):
    ax_id = 0
    # select n_cat point from each category of the testing data

    test_data = np.row_stack((sel0[random.sample(range(np.shape(sel0)[0]),k=n_cat),:],
                            sel1[random.sample(range(np.shape(sel1)[0]),k=n_cat),:],
                            sel2[random.sample(range(np.shape(sel2)[0]),k=n_cat),:],
                            sel3[random.sample(range(np.shape(sel3)[0]),k=n_cat),:]))
    # run for each minimum cluster size
    for clust_id, (min_clust_size, filt_rad) in enumerate(zip(min_cluster_sizes,filter_radius)):
        min_samples = [np.max([1, int(np.round(min_clust_size * min_samples_fact))]) for min_samples_fact in min_samples_facts]
        
        for samp_id, min_sample in enumerate(min_samples):
            
            # go through each method and collect relevant results
            print(run_id, ax_id)
            HDBSCAN_labels = getattr(
                obj, f"HDBSCAN_probabilities_{min_clust_size}_{min_sample}"
            )
            GLOSH_labels = getattr(
                obj, f"HDBSCAN_outlier_scores_{min_clust_size}_{min_sample}"
            )
            LOF_labels = getattr(obj, f"LOF_outlier_scores_{min_clust_size}")
            Mag_labels = getattr(obj, "Mag")
            Med_labels = getattr(obj, f"Mag_off_med_diff_{filt_rad}")

            # get coordinates and offsets for plotting vectors of testing data
            all_lons=getattr(obj,'Lon_off')
            all_lats=getattr(obj,'Lat_off')
            all_X_off=getattr(obj,'X_off')
            all_Y_off=getattr(obj,'Y_off')

            # get testing data
            test_labels = -1 * np.ones(np.shape(test_data[:, 1]))
            mask = (test_data[:, 2] > 1)
            test_labels[mask] = 1  # all inliers
            test_lats = np.empty_like(test_labels)
            test_lons = np.empty_like(test_labels)
            test_X_off = np.empty_like(test_labels)
            test_Y_off = np.empty_like(test_labels)
            for row_id,test_vec in enumerate(test_data):
                
                idx = np.argwhere((r_idx == int(test_vec[0])) & (a_idx == int(test_vec[1])))
                test_lats[row_id] = all_lats[idx[0][0], idx[0][1]]
                test_lons[row_id] = all_lons[idx[0][0], idx[0][1]]
                test_X_off[row_id] = all_X_off[idx[0][0], idx[0][1]]
                test_Y_off[row_id] = all_Y_off[idx[0][0], idx[0][1]]
            
            if plotting == 0:
                fig0, ax0 = plt.subplots(1,1)
                ax0.hist(test_data[:,2],4)
                fig1, ax = plt.subplots(1,1)
                ax.imshow(SHADING,cmap=cm.grayC,alpha=0.5, extent=DEM_EXTENT)
                q = ax.quiver(test_lons,test_lats,
                                test_X_off,test_Y_off,
                                test_data[:,2],
                                scale=50, 
                                width = 0.008, 
                                edgecolor='black',
                                linewidth=0.2)
                plotting = 1

            # collect results of methods for testing data points
            Mag_test_labels = np.full(np.shape(test_labels), np.nan)
            Med_test_labels = np.full(np.shape(test_labels), np.nan)
            HDBSCAN_test_labels = np.full(np.shape(test_labels), np.nan)
            GLOSH_test_labels = np.full(np.shape(test_labels), np.nan)
            LOF_test_labels = np.full(np.shape(test_labels), np.nan)
            for row_id, test_vec in enumerate(test_data):
                idx = np.argwhere((r_idx == int(test_vec[0])) & (a_idx == int(test_vec[1])))
                Mag_test_labels[row_id] = Mag_labels[idx[0][0], idx[0][1]]
                Med_test_labels[row_id] = Med_labels[idx[0][0], idx[0][1]]
                HDBSCAN_test_labels[row_id] = HDBSCAN_labels[idx[0][0], idx[0][1]]
                GLOSH_test_labels[row_id] = GLOSH_labels[idx[0][0], idx[0][1]]
                LOF_test_labels[row_id] = LOF_labels[idx[0][0], idx[0][1]]


            ## convert metric to inlier probability ##

            # magnitude filter (1 - normalised magnitude)
            p_in_mag = 1- (Mag_test_labels-np.nanmin(Mag_labels))/(np.nanmax(Mag_labels)-np.nanmin(Mag_labels))

            # Median filter (1 - max normalised median difference)
            p_in_med = 1- (Med_test_labels/(np.nanmax(Med_labels)))

            # LOF (normalised negative LOF)
            p_in_lof = (LOF_test_labels-np.nanmin(LOF_labels))/(np.nanmax(LOF_labels)-np.nanmin(LOF_labels))

            # HDBSCAN (probability of belonging to cluster (outliers have probability 0))
            p_in_hdb = HDBSCAN_test_labels

            # GLOSH (1 - outlier probability)
            p_in_glsh = 1-GLOSH_test_labels


            # p_in_comb123 = 1 - ((1-p_in_lof)*(1-p_in_glsh)*(1-p_in_hdb))
            # p_in_comb23 = 1 - ((1-p_in_glsh)*(1-p_in_hdb))

            p_in_comb123 = (p_in_lof + p_in_glsh + p_in_hdb)/3
            # p_in_comb23 = (p_in_hdb + p_in_glsh)/2

            p_in_comb23 = (p_in_hdb + p_in_med)/2
            # p_in_comb23 = 1 - ((1-p_in_med)*(1-p_in_hdb))



            # magnitude
            fpr_mag, tpr_mag, thresh_mag = metrics.roc_curve(
                test_labels[~np.isnan(p_in_mag)],
                p_in_mag[~np.isnan(p_in_mag)],
            )
            print(f'thresh mag [0]: {thresh_mag[0]} {thresh_mag[-1]}')
            auc_mag = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_mag)],
                p_in_mag[~np.isnan(p_in_mag)],
            )

            # magnitude
            fpr_med, tpr_med, thresh_med = metrics.roc_curve(
                test_labels[~np.isnan(p_in_med)],
                p_in_med[~np.isnan(p_in_med)],
            )
            print(f'thresh med [0]: {thresh_med[0]} {thresh_med[-1]}')
            auc_med = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_med)],
                p_in_med[~np.isnan(p_in_med)],
            )
            
            # HDBSCAN
            fpr_hdb, tpr_hdb, thresh_hdb = metrics.roc_curve(
                test_labels[~np.isnan(p_in_hdb)],
                p_in_hdb[~np.isnan(p_in_hdb)],
            )
            print(f'thresh hdb [0]: {thresh_hdb[0]} {thresh_hdb[-1]}')
            auc_hdb = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_hdb)],
                p_in_hdb[~np.isnan(p_in_hdb)],
            )
            store_hdb_1 = (fpr_hdb, tpr_hdb, thresh_hdb,auc_hdb)

            # GLOSH
            fpr_glsh, tpr_glsh, thresh_glsh = metrics.roc_curve(
                test_labels[~np.isnan(p_in_glsh)],
                p_in_glsh[~np.isnan(p_in_glsh)],
            )
            print(f'thresh glsh [0]: {thresh_glsh[0]} {thresh_glsh[-1]}')
            auc_glsh = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_glsh)],
                p_in_glsh[~np.isnan(p_in_glsh)],
            )

            # LOF
            fpr_lof, tpr_lof, thresh_lof = metrics.roc_curve(
                test_labels[~np.isnan(p_in_lof)],
                p_in_lof[~np.isnan(p_in_lof)],
            )
            print(f'thresh lof [0]: {thresh_lof[0]} {thresh_lof[-1]}')
            auc_lof = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_lof)],
                p_in_lof[~np.isnan(p_in_lof)],
            )

            # comb123
            fpr_comb123, tpr_comb123, thresh_comb123 = metrics.roc_curve(
                test_labels[~np.isnan(p_in_comb123)],
                p_in_comb123[~np.isnan(p_in_comb123)],
            )
            print(f'thresh comb123 [0]: {thresh_comb123[0]} {thresh_comb123[-1]}')
            auc_comb123 = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_comb123)],
                p_in_comb123[~np.isnan(p_in_comb123)],
            )

            # comb23
            fpr_comb23, tpr_comb23, thresh_comb23 = metrics.roc_curve(
                test_labels[~np.isnan(p_in_comb23)],
                p_in_comb23[~np.isnan(p_in_comb23)],
            )
            print(f'thresh comb23 [0]: {thresh_comb23[0]} {thresh_comb23[-1]}')
            auc_comb23 = metrics.roc_auc_score(
                test_labels[~np.isnan(p_in_comb23)],
                p_in_comb23[~np.isnan(p_in_comb23)],
            )
            

            if verbose:
                print(f'{ax_id} M min, max:{np.min(thresh_mag)},{np.max(thresh_mag)}')
                print(f'{ax_id} Med min, max:{np.min(thresh_med)},{np.max(thresh_med)}')
                print(f'{ax_id} H min, max:{np.min(thresh_hdb)},{np.max(thresh_hdb)}')
                print(f'{ax_id} G min, max:{np.min(thresh_glsh)},{np.max(thresh_glsh)}')
                print(f'{ax_id} L min, max:{np.min(thresh_lof)},{np.max(thresh_lof)}')
                print(f'{ax_id} C123 min, max:{np.min(thresh_comb123)},{np.max(thresh_comb123)}')
                print(f'{ax_id} C23 min, max:{np.min(thresh_comb23)},{np.max(thresh_comb23)}')

                print(f'{ax_id} M min, max:{np.nanmin(p_in_mag)},{np.nanmax(p_in_mag)}')
                print(f'{ax_id} Med min, max:{np.nanmin(p_in_med)},{np.nanmax(p_in_med)}')
                print(f'{ax_id} H min, max:{np.nanmin(p_in_hdb)},{np.nanmax(p_in_hdb)}')
                print(f'{ax_id} G min, max:{np.nanmin(p_in_glsh)},{np.nanmax(p_in_glsh)}')
                print(f'{ax_id} L min, max:{np.nanmin(p_in_lof)},{np.nanmax(p_in_lof)}')
                print(f'{ax_id} C123 min, max:{np.nanmin(p_in_comb123)},{np.nanmax(p_in_comb123)}')
                print(f'{ax_id} C23 min, max:{np.nanmin(p_in_comb23)},{np.nanmax(p_in_comb23)}')

            # set thresholds that are above the maximum value in the test dataset to the max value of the test dataset
            thresh_mag[thresh_mag > np.nanmax(p_in_mag)] = np.nanmax(p_in_mag)
            thresh_med[thresh_med > np.nanmax(p_in_med)] = np.nanmax(p_in_med)
            thresh_hdb[thresh_hdb > np.nanmax(p_in_hdb)] = np.nanmax(p_in_hdb)
            thresh_glsh[thresh_glsh > np.nanmax(p_in_glsh)] = np.nanmax(p_in_glsh)
            thresh_lof[thresh_lof > np.nanmax(p_in_lof)] = np.nanmax(p_in_lof)
            thresh_comb123[thresh_comb123 > np.nanmax(p_in_comb123)] = np.nanmax(p_in_comb123)
            thresh_comb23[thresh_comb23 > np.nanmax(p_in_comb23)] = np.nanmax(p_in_comb23)
            # store true/false positive rate, auc values and thresholds for plotting and calculatign averages
            fpr_stack = [fpr_mag,fpr_med,fpr_hdb,fpr_glsh,fpr_lof,fpr_comb123,fpr_comb23]
            tpr_stack = [tpr_mag,tpr_med,tpr_hdb,tpr_glsh,tpr_lof,tpr_comb123,tpr_comb23]
            auc_stack = [auc_mag,auc_med,auc_hdb,auc_glsh,auc_lof,auc_comb123,auc_comb23]
            thresh_stack = [thresh_mag,thresh_med,thresh_hdb,thresh_glsh,thresh_lof,thresh_comb123,thresh_comb23]
            fpr_list.append(fpr_stack)
            tpr_list.append(tpr_stack)
            thresh_list.append(thresh_stack)
            auc_mag_mat[clust_id,samp_id,run_id] = auc_mag
            auc_med_mat[clust_id,samp_id,run_id] = auc_med
            auc_hdb_mat[clust_id,samp_id,run_id] = auc_hdb
            auc_glsh_mat[clust_id,samp_id,run_id] = auc_glsh
            auc_lof_mat[clust_id,samp_id,run_id] = auc_lof
            auc_comb123_mat[clust_id,samp_id,run_id] = auc_comb123
            auc_comb23_mat[clust_id,samp_id,run_id] = auc_comb23

            # increment ax_id
            ax_id += 1


print(np.shape(fpr_list))        
# get average of auc values across runs
mean_mag = np.mean(auc_mag_mat,axis=2)
mean_med = np.mean(auc_med_mat,axis=2)
mean_hdb = np.mean(auc_hdb_mat,axis=2)
mean_glsh = np.mean(auc_glsh_mat,axis=2)
mean_lof = np.mean(auc_lof_mat,axis=2)
mean_comb123 = np.mean(auc_comb123_mat,axis=2)
mean_comb23 = np.mean(auc_comb23_mat,axis=2)

ax_id = 0
fpr_mag_p = np.array(fpr_list)[:,0]
fpr_med_p = np.array(fpr_list)[:,1]
fpr_hdb_p = np.array(fpr_list)[:,2]
fpr_glsh_p = np.array(fpr_list)[:,3]
fpr_lof_p = np.array(fpr_list)[:,4]
fpr_comb123_p = np.array(fpr_list)[:,5]
fpr_comb23_p = np.array(fpr_list)[:,6]

tpr_mag_p = np.array(tpr_list)[:,0]
tpr_med_p = np.array(tpr_list)[:,1]
tpr_hdb_p = np.array(tpr_list)[:,2]
tpr_glsh_p = np.array(tpr_list)[:,3]
tpr_lof_p = np.array(tpr_list)[:,4]
tpr_comb123_p = np.array(tpr_list)[:,5]
tpr_comb23_p = np.array(tpr_list)[:,6]

thresh_mag_p = np.array(thresh_list,dtype='object')[:,0]
thresh_med_p = np.array(thresh_list,dtype='object')[:,1]
thresh_hdb_p = np.array(thresh_list,dtype='object')[:,2]
thresh_glsh_p = np.array(thresh_list,dtype='object')[:,3]
thresh_lof_p = np.array(thresh_list,dtype='object')[:,4]
thresh_comb123_p = np.array(thresh_list,dtype='object')[:,5]
thresh_comb23_p = np.array(thresh_list,dtype='object')[:,6]



In [ ]:
# make copies of thresh_xxx_p
import copy
thresh_mag_p_cp = copy.deepcopy(thresh_mag_p)
thresh_med_p_cp = copy.deepcopy(thresh_med_p)
thresh_hdb_p_cp = copy.deepcopy(thresh_hdb_p)
thresh_glsh_p_cp = copy.deepcopy(thresh_glsh_p)
thresh_lof_p_cp = copy.deepcopy(thresh_lof_p)
thresh_comb123_p_cp = copy.deepcopy(thresh_comb123_p)
thresh_comb23_p_cp = copy.deepcopy(thresh_comb23_p)
tpr_mag_p_cp = copy.deepcopy(tpr_mag_p)
tpr_med_p_cp = copy.deepcopy(tpr_med_p)
tpr_hdb_p_cp = copy.deepcopy(tpr_hdb_p)
tpr_glsh_p_cp = copy.deepcopy(tpr_glsh_p)
tpr_lof_p_cp = copy.deepcopy(tpr_lof_p)
tpr_comb123_p_cp = copy.deepcopy(tpr_comb123_p)
tpr_comb23_p_cp = copy.deepcopy(tpr_comb23_p)
fpr_mag_p_cp = copy.deepcopy(fpr_mag_p)
fpr_med_p_cp = copy.deepcopy(fpr_med_p)
fpr_hdb_p_cp = copy.deepcopy(fpr_hdb_p)
fpr_glsh_p_cp = copy.deepcopy(fpr_glsh_p)
fpr_lof_p_cp = copy.deepcopy(fpr_lof_p)
fpr_comb123_p_cp = copy.deepcopy(fpr_comb123_p)
fpr_comb23_p_cp = copy.deepcopy(fpr_comb23_p)
sizes = (np.size(min_cluster_sizes),np.size(min_samples_facts),n_runs)
opt_thresh_mag = np.full(sizes,np.nan)
opt_thresh_med = np.full(sizes,np.nan)
opt_thresh_hdb = np.full(sizes,np.nan)
opt_thresh_glsh = np.full(sizes,np.nan)
opt_thresh_lof = np.full(sizes,np.nan)
opt_thresh_comb123 = np.full(sizes,np.nan)
opt_thresh_comb23 = np.full(sizes,np.nan)


In [ ]:

fontsize=16
plt.rcParams.update({'font.size': fontsize})

plot_id = 0
for run_id in range(n_runs):
    ax_id = 0
    for clust_id in range(np.size(min_cluster_sizes)):
        for samp_id in range(np.size(min_samples_facts)):
            print(plot_id)
            print(f'max pos M: {np.argmax(tpr_mag_p_cp[plot_id]-fpr_mag_p_cp[plot_id])} : {thresh_mag_p_cp[plot_id][np.argmax(tpr_mag_p_cp[plot_id]-fpr_mag_p_cp[plot_id])]}')
            print(f'max pos Med: {np.argmax(tpr_med_p_cp[plot_id]-fpr_med_p_cp[plot_id])} : {thresh_med_p_cp[plot_id][np.argmax(tpr_med_p_cp[plot_id]-fpr_med_p_cp[plot_id])]}')
            print(f'max pos H: {np.argmax(tpr_hdb_p_cp[plot_id]-fpr_hdb_p_cp[plot_id])} : {thresh_hdb_p_cp[plot_id][np.argmax(tpr_hdb_p_cp[plot_id]-fpr_hdb_p_cp[plot_id])]}')
            print(f'max pos G: {np.argmax(tpr_glsh_p_cp[plot_id]-fpr_glsh_p_cp[plot_id])} : {thresh_glsh_p_cp[plot_id][np.argmax(tpr_glsh_p_cp[plot_id]-fpr_glsh_p_cp[plot_id])]}')
            print(f'max pos L: {np.argmax(tpr_lof_p_cp[plot_id]-fpr_lof_p_cp[plot_id])} : {thresh_lof_p_cp[plot_id][np.argmax(tpr_lof_p_cp[plot_id]-fpr_lof_p_cp[plot_id])]}')
            print(f'max pos C123: {np.argmax(tpr_comb123_p_cp[plot_id]-fpr_comb123_p_cp[plot_id])} : {thresh_comb123_p_cp[plot_id][np.argmax(tpr_comb123_p_cp[plot_id]-fpr_comb123_p_cp[plot_id])]}')
            print(f'max pos C23: {np.argmax(tpr_comb23_p_cp[plot_id]-fpr_comb23_p_cp[plot_id])} : {thresh_comb23_p_cp[plot_id][np.argmax(tpr_comb23_p_cp[plot_id]-fpr_comb23_p_cp[plot_id])]}')
            opt_thresh_mag[clust_id,samp_id,run_id] = thresh_mag_p_cp[plot_id][np.argmax(tpr_mag_p_cp[plot_id]-fpr_mag_p_cp[plot_id])]
            opt_thresh_med[clust_id,samp_id,run_id] = thresh_med_p_cp[plot_id][np.argmax(tpr_med_p_cp[plot_id]-fpr_med_p_cp[plot_id])]
            opt_thresh_hdb[clust_id,samp_id,run_id] = thresh_hdb_p_cp[plot_id][np.argmax(tpr_hdb_p_cp[plot_id]-fpr_hdb_p_cp[plot_id])]
            opt_thresh_glsh[clust_id,samp_id,run_id] = thresh_glsh_p_cp[plot_id][np.argmax(tpr_glsh_p_cp[plot_id]-fpr_glsh_p_cp[plot_id])]
            opt_thresh_lof[clust_id,samp_id,run_id] = thresh_lof_p_cp[plot_id][np.argmax(tpr_lof_p_cp[plot_id]-fpr_lof_p_cp[plot_id])]
            opt_thresh_comb123[clust_id,samp_id,run_id] = thresh_comb123_p_cp[plot_id][np.argmax(tpr_comb123_p_cp[plot_id]-fpr_comb123_p_cp[plot_id])]
            opt_thresh_comb23[clust_id,samp_id,run_id] = thresh_comb23_p_cp[plot_id][np.argmax(tpr_comb23_p_cp[plot_id]-fpr_comb23_p_cp[plot_id])]
            plot_id +=1
# opt_thresh = [opt_thresh_mag, opt_thresh_hdb, opt_thresh_glsh, opt_thresh_lof]


min_samp_label = [ str(i) for i in min_samples_facts]
min_samp_label[0] = '>0'
# min_clust_label_facts = [str(i) for i in min_clust_facts]
min_clust_label_facts = [str(i) for i in min_cluster_sizes]
min_clust_label = [str(i) for i in min_cluster_sizes]

# print(opt_thresh_glsh[:,:,1])
print(f'max thresh M: {np.max(opt_thresh_mag)}')
print(f'max thresh Med: {np.max(opt_thresh_med)}')
print(f'max thresh H: {np.max(opt_thresh_hdb)}')
print(f'max thresh G: {np.max(opt_thresh_glsh)}')
print(f'max thresh L: {np.max(opt_thresh_lof)}')
print(f'max thresh C123: {np.max(opt_thresh_comb123)}')
print(f'max thresh C23: {np.max(opt_thresh_comb23)}')
print(f'min thresh M: {np.min(opt_thresh_mag)}')
print(f'min thresh Med: {np.min(opt_thresh_med)}')
print(f'min thresh H: {np.min(opt_thresh_hdb)}')
print(f'min thresh G: {np.min(opt_thresh_glsh)}')
print(f'min thresh L: {np.min(opt_thresh_lof)}')
print(f'min thresh C123: {np.min(opt_thresh_comb123)}')
print(f'min thresh C23: {np.min(opt_thresh_comb23)}')

fig, ax = plt.subplots(2,3)

auc0 = ax.flatten()[0].imshow(np.flipud(np.mean(auc_mag_mat,axis=2)),vmin=0.5,vmax=1)
auc0 = ax.flatten()[1].imshow(np.flipud(np.mean(auc_med_mat,axis=2)),vmin=0.5,vmax=1)
auc1 = ax.flatten()[2].imshow(np.flipud(np.mean(auc_lof_mat,axis=2)),vmin=0.5,vmax=1)
auc2 = ax.flatten()[3].imshow(np.flipud(np.mean(auc_hdb_mat,axis=2)),vmin=0.5,vmax=1)
auc3 = ax.flatten()[4].imshow(np.flipud(np.mean(auc_glsh_mat,axis=2)),vmin=0.5,vmax=1)
# auc4 = .flatten()ax[0,4].imshow(np.flipud(np.mean(auc_comb123_mat,axis=2)),vmin=0.5,vmax=1)
auc5 = ax.flatten()[5].imshow(np.flipud(np.mean(auc_comb23_mat,axis=2)),vmin=0.5,vmax=1)

ax.flatten()[0].set_title('Magnitude thresh.')
ax.flatten()[1].set_title('Median difference')
ax.flatten()[2].set_title('LOF')
ax.flatten()[3].set_title('HDBSCAN')
ax.flatten()[4].set_title('GLOSH')
ax.flatten()[5].set_title('COMB median HDBSCAN')

cax = add_right_cax(ax.flatten()[5], pad=0.02, width=0.02)
cbar = fig.colorbar(auc5, cax=cax,label='AUC [-]')
# fig.colorbar(auc5, pad = 0.25,label='AUC [-]')

# for a in ax.ravel():
#     a.set_xticks(np.array(range(len(min_samples_facts))),labels=min_samp_label,rotation=90)
#     a.set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label)

ax[0,0].set_axis_off()

ax[0,1].tick_params(
    axis='x',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
ax[0,1].set_yticks([7,6,5,4,3,2,1,0],labels=['2','3','4','5','6','7','8','9'])

ax[0,2].tick_params(
    axis='x',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
ax[0,2].set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label)

for a in [ax[1,0],ax[1,1],ax[1,2]]:
    a.set_xticks(np.array(range(len(min_samples_facts))),labels=min_samp_label,rotation=90)
    a.set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label)


ax.flatten()[1].set_ylabel('filter radius [pixels]')
ax.flatten()[2].set_ylabel('knn [-]')
ax.flatten()[3].set_ylabel('min. cluster size [-]')
ax.flatten()[4].set_ylabel('min. cluster size [-]')
ax.flatten()[5].set_ylabel('min. cluster size [-]')


ax.flatten()[3].set_xlabel('min. sample fraction')
ax.flatten()[4].set_xlabel('min. sample fraction')
ax.flatten()[5].set_xlabel('min. sample fraction')


fig, ax = plt.subplots(1,6)

auc0 = ax.flatten()[0].imshow(np.flipud(np.mean(auc_mag_mat,axis=2)),vmin=0.5,vmax=1)
auc0 = ax.flatten()[1].imshow(np.flipud(np.mean(auc_med_mat,axis=2)),vmin=0.5,vmax=1)
auc1 = ax.flatten()[2].imshow(np.flipud(np.mean(auc_lof_mat,axis=2)),vmin=0.5,vmax=1)
auc2 = ax.flatten()[3].imshow(np.flipud(np.mean(auc_hdb_mat,axis=2)),vmin=0.5,vmax=1)
auc3 = ax.flatten()[4].imshow(np.flipud(np.mean(auc_glsh_mat,axis=2)),vmin=0.5,vmax=1)
# auc4 = .flatten()ax[0,4].imshow(np.flipud(np.mean(auc_comb123_mat,axis=2)),vmin=0.5,vmax=1)
auc5 = ax.flatten()[5].imshow(np.flipud(np.mean(auc_comb23_mat,axis=2)),vmin=0.5,vmax=1)

ax.flatten()[0].set_title('Magnitude thresh.')
ax.flatten()[1].set_title('Median difference')
ax.flatten()[2].set_title('LOF')
ax.flatten()[3].set_title('HDBSCAN')
ax.flatten()[4].set_title('GLOSH')
ax.flatten()[5].set_title('COMB median HDBSCAN')

cax = add_right_cax(ax.flatten()[5], pad=0.02, width=0.02)
cbar = fig.colorbar(auc5, cax=cax,label='AUC [-]')

ax[0].set_axis_off()
ax[1].tick_params(
    axis='x',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
ax[1].set_yticks([7,6,5,4,3,2,1,0],labels=['2','3','4','5','6','7','8','9'])
ax[2].tick_params(
    axis='x',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
ax[2].set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label)

for a in [ax[3],ax[4],ax[5]]:
    a.set_xticks(np.array(range(len(min_samples_facts))),labels=min_samp_label,rotation=90)
    a.set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label)


ax.flatten()[1].set_ylabel('filter radius [pixels]')
ax.flatten()[2].set_ylabel('knn [-]')
ax.flatten()[3].set_ylabel('minimum cluster size [-]')
ax.flatten()[4].set_ylabel('minimum cluster size [-]')
ax.flatten()[5].set_ylabel('minimum cluster size [-]')


ax.flatten()[3].set_xlabel('min_sample fraction')
ax.flatten()[4].set_xlabel('min_sample fraction')
ax.flatten()[5].set_xlabel('min_sample fraction')
fig.subplots_adjust(left=0.02, bottom=0, right=0.9, top=1, wspace=0.4, hspace=0)


# fig, ax = plt.subplots(4,6)

# auc0 = ax[0,0].imshow(np.flipud(np.mean(auc_mag_mat,axis=2)),vmin=0.5,vmax=1)
# auc0 = ax[0,1].imshow(np.flipud(np.mean(auc_med_mat,axis=2)),vmin=0.5,vmax=1)
# auc1 = ax[0,2].imshow(np.flipud(np.mean(auc_lof_mat,axis=2)),vmin=0.5,vmax=1)
# auc2 = ax[0,3].imshow(np.flipud(np.mean(auc_hdb_mat,axis=2)),vmin=0.5,vmax=1)
# auc3 = ax[0,4].imshow(np.flipud(np.mean(auc_glsh_mat,axis=2)),vmin=0.5,vmax=1)
# # auc4 = ax[0,4].imshow(np.flipud(np.mean(auc_comb123_mat,axis=2)),vmin=0.5,vmax=1)
# auc5 = ax[0,5].imshow(np.flipud(np.mean(auc_comb23_mat,axis=2)),vmin=0.5,vmax=1)

# ax[0,0].set_title('Magnitude thresh.')
# ax[0,1].set_title('Magnitude thresh.')
# ax[0,2].set_title('LOF')
# ax[0,3].set_title('HDBSCAN')
# ax[0,4].set_title('GLOSH')
# # ax[0,4].set_title('COMB123')
# ax[0,5].set_title('COMB23')

# fig.colorbar(auc0, pad = 0.25,label='AUC [-]')
# fig.colorbar(auc1, pad = 0.25,label='AUC [-]')
# fig.colorbar(auc2, pad = 0.25,label='AUC [-]')
# fig.colorbar(auc3, pad = 0.25,label='AUC [-]')
# fig.colorbar(auc4, pad = 0.25,label='AUC [-]')
# fig.colorbar(auc5, pad = 0.25,label='AUC [-]')

# auc_std0 = ax[1,0].imshow(np.flipud(np.std(auc_mag_mat,axis=2)),vmin=0,vmax=0.1)
# auc_std1 = ax[1,1].imshow(np.flipud(np.std(auc_med_mat,axis=2)),vmin=0,vmax=0.1)
# auc_std2 = ax[1,2].imshow(np.flipud(np.std(auc_lof_mat,axis=2)),vmin=0,vmax=0.1)
# auc_std3 = ax[1,3].imshow(np.flipud(np.std(auc_hdb_mat,axis=2)),vmin=0,vmax=0.1)
# auc_std4 = ax[1,4].imshow(np.flipud(np.std(auc_glsh_mat,axis=2)),vmin=0,vmax=0.1)
# # auc_std4 = ax[1,4].imshow(np.flipud(np.std(auc_comb123_mat,axis=2)),vmin=0,vmax=0.1)
# auc_std5 = ax[1,5].imshow(np.flipud(np.std(auc_comb23_mat,axis=2)),vmin=0,vmax=0.1)



# # fig.colorbar(auc_std0, pad = 0.25, label='AUC std. [-]')
# # fig.colorbar(auc_std1, pad = 0.25, label='AUC std. [-]')
# # fig.colorbar(auc_std2, pad = 0.25, label='AUC std. [-]')
# # fig.colorbar(auc_std3, pad = 0.25, label='AUC std. [-]')
# # fig.colorbar(auc_std4, pad = 0.25, label='AUC std. [-]')
# fig.colorbar(auc_std5, pad = 0.25, label='AUC std. [-]')


# thresh0 = ax[2,0].imshow(np.flipud(np.mean(opt_thresh_mag,axis=2)),vmin=0.5,vmax=1)
# thresh1 = ax[2,1].imshow(np.flipud(np.mean(opt_thresh_med,axis=2)),vmin=0.5,vmax=1)
# thresh2 = ax[2,2].imshow(np.flipud(np.mean(opt_thresh_lof,axis=2)),vmin=0.5,vmax=1)
# thresh3 = ax[2,3].imshow(np.flipud(np.mean(opt_thresh_hdb,axis=2)),vmin=0.5,vmax=1)
# thresh4 = ax[2,4].imshow(np.flipud(np.mean(opt_thresh_glsh,axis=2)),vmin=0.5,vmax=1)
# # thresh4 = ax[2,4].imshow(np.flipud(np.mean(opt_thresh_comb123,axis=2)),vmin=0.5,vmax=1)
# thresh5 = ax[2,5].imshow(np.flipud(np.mean(opt_thresh_comb23,axis=2)),vmin=0.5,vmax=1)

# # fig.colorbar(thresh0,pad=0.25,label='inlier prob. [-]')
# # fig.colorbar(thresh1,pad=0.25,label='inlier prob. [-]')
# # fig.colorbar(thresh2,pad=0.25,label='inlier prob. [-]')
# # fig.colorbar(thresh3,pad=0.25,label='inlier prob. [-]')
# # fig.colorbar(thresh4,pad=0.25,label='inlier prob. [-]')
# fig.colorbar(thresh5,pad=0.25,label='inlier prob. [-]')

# std0 = ax[3,0].imshow(np.flipud(np.std(opt_thresh_mag,axis=2)),vmax=0.25)
# std1 = ax[3,1].imshow(np.flipud(np.std(opt_thresh_med,axis=2)),vmax=0.25)
# std2 = ax[3,2].imshow(np.flipud(np.std(opt_thresh_lof,axis=2)),vmax=0.25)
# std3 = ax[3,3].imshow(np.flipud(np.std(opt_thresh_hdb,axis=2)),vmax=0.25)
# std4 = ax[3,4].imshow(np.flipud(np.std(opt_thresh_glsh,axis=2)),vmax=0.25)
# # std4 = ax[3,4].imshow(np.flipud(np.std(opt_thresh_comb123,axis=2)),vmax=0.25)
# std5 = ax[3,5].imshow(np.flipud(np.std(opt_thresh_comb23,axis=2)),vmax=0.25)

# # fig.colorbar(std0,pad=0.25,label='inlier prob. std. [-]')
# # fig.colorbar(std1,pad=0.25,label='inlier prob. std. [-]')
# # fig.colorbar(std2,pad=0.25,label='inlier prob. std. [-]')
# # fig.colorbar(std3,pad=0.25,label='inlier prob. std. [-]')
# # fig.colorbar(std4,pad=0.25,label='inlier prob. std. [-]')
# fig.colorbar(std5,pad=0.25,label='inlier prob. std. [-]')


# ax[0,0].set_ylabel('AUC\nmin_clust | knn')
# ax[1,0].set_ylabel('AUC std.\nmin_clust | knn')
# ax[2,0].set_ylabel('opt. threshold\nmin_clust | knn')
# ax[3,0].set_ylabel('opt. threshold std.\nmin_clust | knn')


# ax[3,0].set_xlabel('min_sample fraction')
# ax[3,1].set_xlabel('min_sample fraction')
# ax[3,2].set_xlabel('min_sample fraction')
# ax[3,3].set_xlabel('min_sample fraction')
# # ax[3,4].set_xlabel('min_sample fraction')
# ax[3,4].set_xlabel('min_sample fraction')
# ax[3,5].set_xlabel('min_sample fraction')

# ax[0,0].set_ylabel('min_cluster_size')

# min_samp_label = [ str(i) for i in min_samples_facts]
# min_samp_label[0] = '>0'
# # min_clust_label_facts = [str(i) for i in min_clust_facts]
# min_clust_label_facts = [str(i) for i in min_cluster_sizes]
# min_clust_label = [str(i) for i in min_cluster_sizes]

# for a in ax.ravel():
#     a.set_xticks(np.array(range(len(min_samples_facts))),labels=min_samp_label,rotation=90)
#     a.set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label)
#     # secay = a.secondary_yaxis('right')
#     # secay.set_ylabel('* N_{overlap}')
#     # secay.set_yticks(np.flipud(range(len(min_cluster_sizes))),labels=min_clust_label_facts)


# fig.tight_layout()

In [9]:
import copy
import matplotlib.pyplot as plt
import numpy as np
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec

from cmcrameri import cm

import pyproj
from pyproj import CRS

def plot_vec_attr_alpha(obj,attr,step,scale,width =0.005 ,attr_lims=[0,1],qk_length=1,shading = [],dem_extent = [],lat_lims = [],lon_lims = [],alpha=0,fig_ax= []):
    """
    Plots displacement as vectors in slantrange - azimuth plane and 
    assigns colour based on attribute value and limits

    Args:
        obj (class object): _description_
        attr (str): _description_
        step (int): _description_
        scale (int): _description_
        attr_lims (list of floats, optional): _description_. Defaults to [0,1]
        qk_length (float, optional): _description_. Defaults to 1
        shading (np.array, optional): _description_. Defaults to [] -> do not use.
        dem_extent (list, optional): _description_. Defaults to [] -> do not use.
        lat_lims (list, optional): _description_. Defaults to [] -> do not use.
        lon_lims (list, optional): _description_. Defaults to [] -> do not use.
    """
    # get length of 1 deg. for scalebar

    distance_meters = sm.plot.get_1deg_dist()

    # copy data to manipulate limits without messing with the original data
    if attr != []:
        attr_copy = copy.deepcopy(getattr(obj,attr))

        # change limits for nicer plotting
        alpha_map = np.ones_like(attr_copy)
        alpha_map[attr_copy<attr_lims[0]] = alpha
        attr_copy[attr_copy<attr_lims[0]] = attr_lims[0]
        attr_copy[attr_copy>attr_lims[1]] = attr_lims[1]
        

    if lat_lims == []:
        lat_lims = [np.min(obj.Lat_off_vec),np.max(obj.Lat_off_vec)]
    
    if lon_lims == []:
        lon_lims = [np.min(obj.Lon_off_vec),np.max(obj.Lon_off_vec)]

    # plotting
    if fig_ax == []:
        fig1, ax = plt.subplots(1,1,figsize=(8,8))
    else:
        fig1 = fig_ax[0]
        ax = fig_ax[1]
        
    if (dem_extent != []):
        ax.imshow(shading,cmap=cm.grayC,alpha=0.5, extent=dem_extent)
    
    if attr == []:
        q = ax.quiver(obj.Lon_off[::step,::step],obj.Lat_off[::step,::step],
                    obj.X_off[::step,::step],obj.Y_off[::step,::step],
                    color='black',
                    alpha=alpha_map[::step,::step],
                    scale=scale, 
                    width = width, 
                    edgecolor='black',
                    linewidth=0.2,)
    else:
        q = ax.quiver(obj.Lon_off[::step,::step],obj.Lat_off[::step,::step],
                        obj.X_off[::step,::step],obj.Y_off[::step,::step],
                        attr_copy[::step,::step],
                        alpha=alpha_map[::step,::step],
                        scale=scale, 
                        width = width, 
                        edgecolor='black',
                        linewidth=0.2,
                        clim=attr_lims)
    ax.set_ylim(lat_lims)
    ax.set_xlim(lon_lims)
    ax.add_artist(ScaleBar(distance_meters,location='lower right'))
    # ax.add_artist(ScaleBar(distance_meters,location='lower right',font_properties={'size': 20}))
    # fig1.colorbar(q,ax=axes,extend='both')
    # cax = add_right_cax(ax, pad=0.02, width=0.02)
    # cbar = fig.colorbar(q, cax=cax,label='inlier probability [-]')
    ax.set_axis_off()
    # ax.set_yticks([-7.55,-7.54,-7.53])
    # ax.set_xticks([110.43, 110.44, 110.45])
    # ax.set_xticklabels(['110.43','110.44','110.45'])
    # ax.set_xlabel(r'Longitude [$^\circ$E]')
    # ax.set_ylabel(r'Latitude [$^\circ$N]')
    qk = ax.quiverkey(q,
                             0.5,
                             0.95,
                             qk_length,
                             str(qk_length) +' m displacement in slant range–azimuth plane',
                             labelpos = 'E',
                             coordinates='figure')
    # if attr != []:
    #     ax.set_title(f'{attr}: min = {np.nanmax([np.nanmin(attr_copy),attr_lims[0]]):1.3f} max = {np.nanmin([np.nanmax(attr_copy),attr_lims[1]]):1.3f}')
    return q

In [17]:
%matplotlib osx

fig, ax = plt.subplots(1,1,figsize=(8,8))
alpha = 0.3
q1 = plot_vec_attr_alpha(example_pairs[0].Stack[2],'Mag',5,50,0.01,[0,1],5,SHADING,DEM_EXTENT,[-7.565, -7.515],[110.415,110.468],alpha=alpha,fig_ax=(fig,ax))


# fig, ax = plt.subplots(1,1,figsize=(8,8))
sm.plot.plot_ra_offsets(example_pairs[0].Stack[2],SHADING,DEM_EXTENT,[-1,1],grid_size=1000,lat_lims=[-7.565, -7.515],lon_lims=[110.415,110.468],cmap=cm.vik)


: 

In [ ]:
%matplotlib osx
# import matplotlib as mpl
# mpl.rcParams.update(mpl.rcParamsDefault)
# mpl.rcParams.update({'font.size': 14})

fig, ax = plt.subplots(3,5)
alpha =0.3

for i, obj in enumerate(example_pairs[0].Stack):

    obj_cp = copy.deepcopy(obj)

    # get attr.
    mag = getattr(obj_cp,'Mag')
    med_diff = getattr(obj_cp,'Mag_off_med_diff_7')
    LOF_scores = getattr(obj_cp, f"LOF_outlier_scores_300")
    HDBSCAN_labels = getattr(obj_cp, f"HDBSCAN_probabilities_300_1")
    HDBSCAN_labels2 = getattr(obj_cp, f"HDBSCAN_probabilities_300_30")
    GLOSH_labels = getattr(obj_cp, f"HDBSCAN_outlier_scores_300_1")

    # calculate probablities
    p_in_mag = 1 - (mag-np.nanmin(mag))/(np.nanmax(mag)-np.nanmin(mag))
    p_in_med = 1 - med_diff/(np.nanmax(med_diff))
    p_in_LOF = (LOF_scores-np.nanmin(LOF_scores))/(np.nanmax(LOF_scores)-np.nanmin(LOF_scores))
    p_in_HDBSCAN = HDBSCAN_labels
    p_in_HDBSCAN2 = HDBSCAN_labels2
    p_in_GLOSH = 1 - GLOSH_labels
    p_in_comb23 = (p_in_HDBSCAN+p_in_med)/2 

    # assign as attributes
    setattr(obj_cp,'p_in_mag',p_in_mag)
    setattr(obj_cp,'p_in_med',p_in_med)
    setattr(obj_cp,'p_in_LOF',p_in_LOF)
    setattr(obj_cp,'p_in_HDBSCAN',p_in_HDBSCAN)
    setattr(obj_cp,'p_in_HDBSCAN2',p_in_HDBSCAN2)
    setattr(obj_cp,'p_in_GLOSH',p_in_GLOSH)
    setattr(obj_cp,'p_in_comb23',p_in_comb23)

    q1 = plot_vec_attr_alpha(obj_cp,'p_in_med',5,50,0.01,[0,1],5,SHADING,DEM_EXTENT,[-7.545, -7.535],[110.435,110.448],alpha=alpha,fig_ax=(fig,ax[0,i]))
    q2 = plot_vec_attr_alpha(obj_cp,'p_in_HDBSCAN',5,50,0.01,[0,1],5,SHADING,DEM_EXTENT,[-7.545, -7.535],[110.435,110.448],alpha=alpha,fig_ax=(fig,ax[1,i]))
    q3 = plot_vec_attr_alpha(obj_cp,'p_in_comb23',5,50,0.01,[0,1],5,SHADING,DEM_EXTENT,[-7.545, -7.535],[110.435,110.448],alpha=alpha,fig_ax=(fig,ax[2,i]))

a=1
cax = add_right_cax(ax[0,4], pad=0.02, width=0.02)
cbar = fig.colorbar(q1, cax=cax,label='inlier probability [-]')
cax = add_right_cax(ax[1,4], pad=0.02, width=0.02)
cbar = fig.colorbar(q2, cax=cax,label='inlier probability [-]')
cax = add_right_cax(ax[2,4], pad=0.02, width=0.02)
cbar = fig.colorbar(q3, cax=cax,label='inlier probability [-]')
plt.show()

# fig.tight_layout()

In [ ]:
%matplotlib osx
# import matplotlib as mpl
# mpl.rcParams.update(mpl.rcParamsDefault)
# mpl.rcParams.update({'font.size': 14})

fig, ax = plt.subplots(2,3)
alpha =0.3

for i, (obj,ax_i) in enumerate(zip(example_pairs[0].Stack,ax.ravel())):

    obj_cp = copy.deepcopy(obj)

    # get attr.
    med_diff = getattr(obj_cp,'Mag_off_med_diff_7')


    # calculate probablities
    p_in_med = 1 - med_diff/(np.nanmax(med_diff))

    # assign as attributes
    setattr(obj_cp,'p_in_med',p_in_med)


    q1 = plot_vec_attr_alpha(obj_cp,'p_in_med',5,50,0.01,[0.9,1],5,SHADING,DEM_EXTENT,[-7.545, -7.535],[110.435,110.448],alpha=alpha,fig_ax=(fig,ax_i))

a=1
cax = add_right_cax(ax[1,1], pad=0.02, width=0.02)
cbar = fig.colorbar(q1, cax=cax,label='inlier probability [-]')
ax[1,2].set_axis_off()
plt.show()
# fig.tight_layout()

In [ ]:
a=1

In [ ]:

plt.close('all')

In [ ]:
# lava flow files:
L1888_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1888_v2.shp'
L1992_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1992.shp'
L1998_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/L1998.shp'
CRATER_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/merapi_maps/Merapi_crater.shp'

# Read the shapefile
L1888 = gpd.read_file(L1888_FILE)
L1992 = gpd.read_file(L1992_FILE)
L1998 = gpd.read_file(L1998_FILE)
CRATER = gpd.read_file(CRATER_FILE)

# Extract latitude and longitude into separate columns
coords_L1888 = np.array(list(L1888["geometry"][0].coords))
coords_L1992 = np.array(list(L1992["geometry"][0].coords))
coords_L1998 = np.array(list(L1998["geometry"][0].coords))
coords_CRATER = np.array(list(CRATER["geometry"][0].coords))

In [ ]:
%matplotlib osx
lon_lims = [110.428, 110.45]
lat_lims = [-7.550, -7.528]
# import cmcrameri as cm

def plot_ra_offsets(obj,shading,dem_extent,clims,grid_size=1000,lat_lims=[],lon_lims=[],cmap=cm.vik):
    # get length of 1 deg. for scalebar
    distance_meters = sm.plot.get_1deg_dist()
        # change limits for nicer plotting
    R_off = getattr(obj,'R_off')
    A_off = getattr(obj,'A_off')
    Lon_off = getattr(obj,'Lon_off')
    Lat_off = getattr(obj,'Lat_off')
    

    if lat_lims == []:
        lat_lims = [np.min(Lat_off),np.max(Lat_off)]
    
    if lon_lims == []:
        lon_lims = [np.min(Lon_off),np.max(Lon_off)]

    # plotting
    fig1, axes = plt.subplots(2,1,figsize=(8,8))
    axes[0].imshow(shading,cmap=cm.grayC,alpha=0.5, extent=dem_extent)
    axes[1].imshow(shading,cmap=cm.grayC,alpha=0.5, extent=dem_extent)

    plot_data = axes[0].hexbin(Lon_off.flatten(),Lat_off.flatten(),C=R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
    axes[0].set_xlim(lon_lims)
    axes[0].set_ylim(lat_lims)
    axes[0].set_aspect('equal', 'box')
    axes[0].add_artist(ScaleBar(distance_meters,location='lower left'))
    # axes[0].set_axis_off()
    cbar = plt.colorbar(plot_data,ax=axes[0])
    cbar.set_label('Slant range offset [m]')

    plot_data = axes[1].hexbin(Lon_off.flatten(),Lat_off.flatten(),C=A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
    axes[1].set_xlim(lon_lims)
    axes[1].set_ylim(lat_lims)
    axes[1].set_aspect('equal', 'box')
    axes[1].add_artist(ScaleBar(distance_meters,location='lower left'))
    # axes[0].set_axis_off()
    cbar = plt.colorbar(plot_data,ax=axes[1])
    cbar.set_label('Azimuth offset [m]')
    return fig1, axes

fig, ax = plot_ra_offsets(example_pairs[0].Stack[0],SHADING,DEM_EXTENT,[-3,3],500,lat_lims=lat_lims,lon_lims=lon_lims,cmap=cm.vik)
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[0].plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
ax[0].plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:pink',label='L1992')
ax[0].plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
ax[0].plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
ax[0].legend(loc='upper left')
ax[1].plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue')
ax[1].plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:pink')
ax[1].plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green')
ax[1].plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--')

In [ ]:
plt.close(all)

In [ ]:
## sequential median (0.9) then 

cut_off_med = 0.9


for obj in example_pairs[0].Stack:
    print(obj)
    obj.rem_outliers_median(7,cut_off_med)
    obj.reset_vecs()



In [ ]:
## plot outlier removed maps
lon_lims = [110.428, 110.45]
lat_lims = [-7.550, -7.528]
fig, ax = plot_ra_offsets(example_pairs[0].Stack[4],SHADING,DEM_EXTENT,[-3,3],500,lat_lims=lat_lims,lon_lims=lon_lims,cmap=cm.vik)
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[0].plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
ax[0].plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:pink',label='L1992')
ax[0].plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
ax[0].plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
ax[1].plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue')
ax[1].plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:pink')
ax[1].plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green')
ax[1].plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--')

In [ ]:
def Run_MKA(q_obj,indeces=[],window_size=1,comp_lim=0.5,CI_lim=5):
        # """
        # Run Multi-kernel averaging where user can define seleced indices from the stack, 
        # desired window size, and a completion factor as a high pass filter

        # Args:
        #     indices (list, optional): indices of slices from datastack used for MKA. Defaults to [], use all data.
        #     window_size (int, optional): window dimension for MKA, odd numbers 
        #                                  prefered because of pixel centering. 
        #                                  Defaults to 1.
        #     comp_lim (float, optional): completion limit between [0.0, 1.0]
        #                                 Only take data for MKA if more than 
        #                                 comp_lim of the stack is not nan. 
        #                                 Defaults to 0.5.

        # Returns:
        #     avg_map: Multi-kernel Average map
        # """
        # get stack data
        if indeces==[]:
            stack_R = [obj.R_off for obj in q_obj.Stack]
            stack_A = [obj.A_off for obj in q_obj.Stack]
            stack_ccp = [obj.Ccp_off for obj in q_obj.Stack]
            stack_ccs = [obj.Ccs_off for obj in q_obj.Stack]
        else:
            substack = [q_obj.Stack[i] for i in indeces]
            stack_R = np.stack([obj.R_off for obj in substack],axis=0)
            stack_A = np.stack([obj.A_off for obj in substack],axis=0)
            stack_ccp = np.stack([obj.Ccp_off for obj in substack],axis=0)
            stack_ccs = np.stack([obj.Ccs_off for obj in substack],axis=0)

        # create list of maps to make 
        avg_maps = []

        for stack in [stack_R,stack_A]:
            # set window size according to stack dimensions
            window_shape = (stack.shape[0], window_size, window_size)
            print(np.shape(window_shape))
            # use np.lib.stride_tricks.sliding_window_view to devided data into windows
            # 1.2xfaster than sklearn view_as_windows
            win_data = np.lib.stride_tricks.sliding_window_view(stack, window_shape)[0]

            # remove data that is nan for too many different window sizes
            nan_frac = np.sum(np.isnan(win_data), axis=2) / (window_size ** 2)
            nan_frac = nan_frac/np.shape(win_data)[2]
            nan_frac[nan_frac > comp_lim] = np.nan
            nan_frac[nan_frac <= comp_lim] = 1
            win_data = np.multiply(win_data, nan_frac[..., np.newaxis])

            # define shape of multi-kernel averaged map (same as input data), filled with nan
            Avg_map = np.full(stack.shape[1:], np.nan)

            # per window, go take 95% confidence interval data and take average (mean)
            for win_i in range(win_data.shape[0]):
                if win_i % 50 == 0:
                    print('win_i', win_i)
                for win_j in range(win_data.shape[1]):
                    offset = window_size // 2
                    # extract relevant window
                    win = win_data[win_i, win_j]
                    # calculate 95 % confidence interval
                    if np.size(win)==1:
                         Avg_map[win_i + offset, win_j + offset] = np.nanmean(win)
                    elif CI_lim == 0:
                         Avg_map[win_i + offset, win_j + offset] = np.nanmean(win)
                    else:
                        percentiles = np.nanpercentile(win, [CI_lim/2, 100-(CI_lim/2)])
                        # mask data outside 95% confidence interval with nan
                        mask = (win < percentiles[0]) | (win > percentiles[1])
                        win[mask] = np.nan
                        # calculate mean of window (offset by floor(window_size/2) because of border)
                        
                        Avg_map[win_i + offset, win_j + offset] = np.nanmean(win)
            avg_maps.append(Avg_map)
        q_obj.MKA_R_off = avg_maps[0]
        q_obj.MKA_A_off = avg_maps[1]
        # stack_obj.MKA_Ccp_off = avg_maps[2]
        # stack_obj.MKA_Ccs_off = avg_maps[3]

        return q_obj.MKA_R_off, q_obj.MKA_A_off

In [ ]:
indeces = [0,1,2,3,4]
MKA_R_off, MKA_A_off = Run_MKA(example_pairs[0],indeces,1,1,5)

In [ ]:
# # MKA reuslt
# query points
q1 = [110.446216,-7.536389] # stable no veg north
q2 = [110.42647,-7.53297] # stable veg north west
q3 = [110.4421429,-7.5377992] # L1888
q4 = [110.44214,-7.54237] # L1998 large disp
q5 = [110.44013,-7.54678] # stable no veg south
q6 = [110.44200,-7.53604] # L1956 offset gradient
query_points = np.stack((q1,q2,q3,q4,q5,q6))
r = 50

[q_mean_MKA, q_median_MKA, q_std_MKA, q_95_MKA], coordinate_circles = example_pairs[0].query_point_MKA('MKA_R_off',
                                                                                                 query_points[:,1],
                                                                                                 query_points[:,0],
                                                                                                 r)  
print('\n')
print(f'Range mean for A, B, C: {q_mean_MKA}')
print(f'Range median for A, B, C: {q_median_MKA}')
print(f'Range standard deviation for A, B, C: {q_std_MKA}')
print(f'Range 95\% confidence interval fro A, B, C: {q_95_MKA}')

[q_mean_MKA, q_median_MKA, q_std_MKA, q_95_MKA], coordinate_circles = example_pairs[0].query_point_MKA('MKA_A_off',
                                                                                                 query_points[:,1],
                                                                                                 query_points[:,0],
                                                                                                 r)  
print('\n')
print(f'Azimuth mean for A, B, C: {q_mean_MKA}')
print(f'Azimuth median for A, B, C: {q_median_MKA}')
print(f'Azimuth standard deviation for A, B, C: {q_std_MKA}')
print(f'Azimuth 95\% confidence interval fro A, B, C: {q_95_MKA}')


In [ ]:
import copy
import matplotlib.pyplot as plt
import numpy as np
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec

from cmcrameri import cm

import pyproj
from pyproj import CRS
def plot_ra_offsets_MKA(obj,MKA_R_off,MKA_A_off,shading,dem_extent,clims,grid_size=1000,lat_lims=[],lon_lims=[],cmap=cm.vik,width=[]):
    # get length of 1 deg. for scalebar
    distance_meters = sm.plot.get_1deg_dist()
        # change limits for nicer plotting
    if width==[]:
        Lon_off = getattr(obj.Stack[0],'Lon_off')
        Lat_off = getattr(obj.Stack[0],'Lat_off')
    else:
        obj.get_latlon_from_file(width)
        obj.add_lat_lon_to_data(R_START,A_START)
        obj.crop_stack_ccs(R_STEP,A_STEP)
    

    rng = np.linspace(obj.Limits[0],obj.Limits[1],int((obj.Limits[1]-obj.Limits[0])/R_STEP)+1)
    azi = np.linspace(obj.Limits[2],obj.Limits[3],int((obj.Limits[3]-obj.Limits[2])/A_STEP)+1)
    RNG, AZI = np.meshgrid(rng,azi)


    # find number of range estimates
    for d in obj.Mask_data:
        Lat_off = np.full(np.shape(RNG)[::-1], np.nan)
        Lon_off = np.full(np.shape(RNG)[::-1], np.nan)

        # fill arrays from indexes 
        Lat_off[d[11],d[12]] = d[9]
        Lon_off[d[11],d[12]] = d[10]

    if lat_lims == []:
        lat_lims = [np.min(Lat_off),np.max(Lat_off)]
    
    if lon_lims == []:
        lon_lims = [np.min(Lon_off),np.max(Lon_off)]

    # plotting
    fig1, axes = plt.subplots(2,1,figsize=(8,8))
    axes[0].imshow(shading,cmap=cm.grayC,alpha=0.5, extent=dem_extent)
    axes[1].imshow(shading,cmap=cm.grayC,alpha=0.5, extent=dem_extent)

    plot_data = axes[0].hexbin(Lon_off.flatten(),Lat_off.flatten(),C=MKA_R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
    axes[0].set_xlim(lon_lims)
    axes[0].set_ylim(lat_lims)
    axes[0].set_aspect('equal', 'box')
    axes[0].add_artist(ScaleBar(distance_meters,location='lower left'))
    # axes[0].set_axis_off()
    cbar = plt.colorbar(plot_data,ax=axes[0])
    cbar.set_label('Slant range offset [m]')

    plot_data = axes[1].hexbin(Lon_off.flatten(),Lat_off.flatten(),C=MKA_A_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
    axes[1].set_xlim(lon_lims)
    axes[1].set_ylim(lat_lims)
    axes[1].set_aspect('equal', 'box')
    axes[1].add_artist(ScaleBar(distance_meters,location='lower left'))
    # axes[0].set_axis_off()
    cbar = plt.colorbar(plot_data,ax=axes[1])
    cbar.set_label('Azimuth offset [m]')
    return fig1, axes
    

In [ ]:
# get MKA vec data

def get_MKA_vec(obj):
    Lon_off = getattr(obj.Stack[0],'Lon_off')
    Lat_off = getattr(obj.Stack[0],'Lat_off')

    rng = np.linspace(obj.Limits[0],obj.Limits[1],int((obj.Limits[1]-obj.Limits[0])/R_STEP)+1)
    azi = np.linspace(obj.Limits[2],obj.Limits[3],int((obj.Limits[3]-obj.Limits[2])/A_STEP)+1)
    RNG, AZI = np.meshgrid(rng,azi)

    # find number of range estimates
    for d in obj.Mask_data:
        Lat_off = np.full(np.shape(RNG)[::-1], np.nan)
        Lon_off = np.full(np.shape(RNG)[::-1], np.nan)

        # fill arrays form indexes 
        Lat_off[d[11],d[12]] = d[9]
        Lon_off[d[11],d[12]] = d[10]
    setattr(obj,'Lon_off_MKA',Lon_off)
    setattr(obj,'Lat_off_MKA',Lat_off)
    
    nan_mask = np.isnan(obj.MKA_R_off)
    MKA_R_off_vec = obj.MKA_R_off[~nan_mask].ravel()
    MKA_A_off_vec = obj.MKA_A_off[~nan_mask].ravel()
    Lon_off_MKA_vec = obj.Lon_off_MKA[~nan_mask].ravel()
    Lat_off_MKA_vec = obj.Lat_off_MKA[~nan_mask].ravel()

    # set attributes
    setattr(obj,'MKA_R_off_vec', MKA_R_off_vec)
    setattr(obj,'MKA_A_off_vec', MKA_A_off_vec)
    setattr(obj,'Lon_off_MKA_vec', Lon_off_MKA_vec)
    setattr(obj,'Lat_off_MKA_vec', Lat_off_MKA_vec)

    return MKA_R_off_vec, MKA_A_off_vec, Lon_off_MKA_vec, Lat_off_MKA_vec

# get_MKA_vec(example_pairs[0])


In [ ]:
%matplotlib osx
## plot MKA results
lon_lims = [110.428, 110.45]
lat_lims = [-7.550, -7.528]



fig, ax = plot_ra_offsets_MKA(example_pairs[0],MKA_R_off,MKA_A_off,SHADING,DEM_EXTENT,[-3,3],500,lat_lims=lat_lims,lon_lims=lon_lims,cmap=cm.vik,width=WIDTH)
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[0].plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
ax[0].plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:pink',label='L1992')
ax[0].plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
ax[0].plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
ax[1].plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue')
ax[1].plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:pink')
ax[1].plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green')
ax[1].plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--')

In [ ]:
%matplotlib osx
# swath profile simple: 

from functools import partial
from mpl_point_clicker import clicker # for user input from map
from inpoly import inpoly2 # for fast inpolygon checks

from matplotlib.ticker import FormatStrFormatter
from shapely.geometry import Polygon, LineString
# from mpl_interactions import zoom_factory, panhandler


# define variables:

grid_size = 1000
cmap = cm.vik
clims= [-3,3]
lon_lims = [110.428, 110.45]
lat_lims = [-7.550, -7.528]
# create swath profile from user input


def poly_line_buffer_lat_lon(line_coords,width):
    line = LineString(line_coords)
    local_azimuthal_projection = f"+proj=aeqd +R=6371000 +units=m +lat_0={line.coords[0][1]} +lon_0={line.coords[0][0]}"

    wgs84_to_aeqd = partial(
            pyproj.transform,
            pyproj.Proj('+proj=longlat +datum=WGS84 +no_defs'),
            pyproj.Proj(local_azimuthal_projection),
            )

    aeqd_to_wgs84 = partial(
            pyproj.transform,
            pyproj.Proj(local_azimuthal_projection),
            pyproj.Proj('+proj=longlat +datum=WGS84 +no_defs'),
            )
    line_transformed = transform(wgs84_to_aeqd, line)
    buffer_line = line_transformed.buffer(width/2,cap_style=2)# capstyle 2 for flat caps the meet profile (so do not go beyond the line)
    return transform(aeqd_to_wgs84, buffer_line)




fig, ax0 =plt.subplots(constrained_layout=True, figsize=(15,10))
plot_data = ax0.hexbin(example_pairs[0].Stack[0].Lon_off.flatten(),example_pairs[0].Stack[0].Lat_off.flatten(),C=MKA_R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax0.set_xlim(lon_lims)
ax0.set_ylim(lat_lims)
ax0.set_aspect('equal', 'box')
ax0.add_artist(ScaleBar(110123.8348,location='lower left'))
ax0.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
ax0.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:pink',label='L1992')
ax0.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
ax0.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
ax0.set_axis_off()
# axes[0].set_axis_off()
cbar = plt.colorbar(plot_data,ax=ax0)
cbar.set_label('Slant range offset [m]')
klicker = clicker(ax0, ["query_point"], markers=["x"], colors='green',markersize=15,markeredgewidth=3,**{"linestyle": "-"})









In [ ]:
from shapely.ops import transform

width_swath = 50 # m
mouse_pos = klicker.get_positions()['query_point']
start_base = mouse_pos[-2,:] # starting point for profile baseline
end_base = mouse_pos[-1,:] # end point for profile baseline
print('query line: ', start_base, end_base, '\n width: ' , width_swath, ' m')

# start_base = [110.4405519,-7.5337943] # starting point for profile baseline
# end_base = [110.4442428,-7.5371901] # end point for profile baseline

# get line coords and add buffer
line_coords = np.asarray([start_base,end_base])
buffer_poly = poly_line_buffer_lat_lon(line_coords,width_swath)


# show line with buffer
fig, ax0 =plt.subplots(constrained_layout=True, figsize=(15,10))
plot_data = ax0.hexbin(example_pairs[0].Stack[0].Lon_off.flatten(),example_pairs[0].Stack[0].Lat_off.flatten(),C=MKA_R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax0.set_xlim(lon_lims)
ax0.set_ylim(lat_lims)
ax0.set_aspect('equal', 'box')
ax0.add_artist(ScaleBar(110123.8348,location='lower left'))
# axes[0].set_axis_off()
cbar = plt.colorbar(plot_data,ax=ax0)
cbar.set_label('Range offset [m]')
ax0.plot(np.asarray(buffer_poly.exterior.coords)[:,0],np.asarray(buffer_poly.exterior.coords)[:,1])
ax0.plot(np.asarray(line_coords)[:,0],np.asarray(line_coords[:,1]))

In [ ]:
plt.close('all')

In [ ]:
import contextlib
import shapely
import warnings
from distutils.version import LooseVersion

SHAPELY_GE_20 = str(shapely.__version__) >= LooseVersion("2.0")

try:
    from shapely.errors import ShapelyDeprecationWarning as shapely_warning
except ImportError:
    shapely_warning = None

if shapely_warning is not None and not SHAPELY_GE_20:
    @contextlib.contextmanager
    def ignore_shapely2_warnings():
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=shapely_warning)
            yield
else:
    @contextlib.contextmanager
    def ignore_shapely2_warnings():
        yield

In [ ]:
# get all points within polygon
from skspatial.objects import Line as sksline
import scipy.spatial.transform 
line = sksline.from_points(point_a=start_base, point_b=end_base)

def geodetic2enu(lat, lon, alt, lat_org, lon_org, alt_org):
    transformer = pyproj.Transformer.from_crs(
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
        )
    x, y, z = transformer.transform( lon,lat,  alt,radians=False)
    x_org, y_org, z_org = transformer.transform( lon_org,lat_org,  alt_org,radians=False)
    vec=np.array([[ x-x_org, y-y_org, z-z_org]]).T
    
    rot1 =  scipy.spatial.transform.Rotation.from_euler('x', -(90-lat_org), degrees=True).as_matrix()#angle*-1 : left handed *-1
    rot3 =  scipy.spatial.transform.Rotation.from_euler('z', -(90+lon_org), degrees=True).as_matrix()#angle*-1 : left handed *-1
    rotMatrix = rot1.dot(rot3)  
    
    enu = np.squeeze(rotMatrix.dot(vec).T)
    return enu


isin_list = []
proj_point_list = []
obj_list = [obj for obj in example_pairs[0].Stack]
obj_list.append(MKA_data_list[0])
obj_list.append(MKA_data_list[1])
print(obj_list)
lon_off_vec_list = ['Lon_off_vec','Lon_off_vec','Lon_off_vec','Lon_off_vec','Lon_off_vec','Lon_off_MKA_vec','Lon_off_MKA_vec']
lat_off_vec_list = ['Lat_off_vec','Lat_off_vec','Lat_off_vec','Lat_off_vec','Lat_off_vec','Lat_off_MKA_vec','Lat_off_MKA_vec']
R_off_vec_list = ['R_off_vec','R_off_vec','R_off_vec','R_off_vec','R_off_vec','MKA_R_off_vec','MKA_R_off_vec']
A_off_vec_list = ['A_off_vec','A_off_vec','A_off_vec','A_off_vec','A_off_vec','MKA_A_off_vec','MKA_A_off_vec']


for (obj,lon_off_name,lat_off_name,R_off_name) in zip(obj_list,lon_off_vec_list,lat_off_vec_list,R_off_vec_list):
    lons = getattr(obj,lon_off_name)
    lats = getattr(obj,lat_off_name)
    lon_lat = np.dstack((lons,lats))
    isin, ison = inpoly2(np.squeeze(lon_lat),np.asarray(buffer_poly.exterior.coords))
    print(np.count_nonzero(isin))
    isin_list.append(isin)
    proj_point = np.empty(shape=(np.count_nonzero(isin), 3), dtype=float) # pre-alloc nd-array                                  0)

    with ignore_shapely2_warnings():
        for p_id, lon_lat_id in enumerate(zip(np.squeeze(lon_lat)[isin,:],getattr(obj,R_off_name)[isin])):
            point = lon_lat_id[0]
            if np.mod(p_id,1000)==0:
                    print('fraction complete:',p_id/np.count_nonzero(isin)) # progress tracker
            proj_point[p_id,:]=np.append(np.asarray(line.project_point(point)),lon_lat_id[1])
    

    # transform projected points to east,north,(up)
    point_transformed = geodetic2enu(proj_point[:,1],
                                    proj_point[:,0],
                                    np.squeeze(np.zeros((np.count_nonzero(isin),1))).T,
                                    start_base[1],
                                    start_base[0],
                                    0)
    #lon,lat,dist along profile, disp
    proj_point = np.stack((proj_point[:,0], 
                                proj_point[:,1],
                                np.linalg.norm(point_transformed[:,0:2],axis=1),
                                proj_point[:,2])).T
    print(proj_point)
    proj_point = proj_point[~np.isnan(proj_point[:,3])]
    proj_point_list.append(proj_point)
    
# proj_points=line.project_point(point(np.squeeze(lon_lat)[isin,:]))

In [ ]:
print(proj_point_list[2])

fig, ax = plt.subplots(1,1)

ax.scatter(proj_point_list[2][:,2],proj_point_list[2][:,3])
# ax.set_aspect('equal')

In [ ]:
textsize = 15
plt.rc('font', size=textsize) 
# %matplotlib inline
# bin profile output in sections with width xd to calculate percentiles
dx = 10
# offset = 0.2
grid_size = 1000
clims = [-3, 3]
lon_lims = [110.436, 110.445]
lat_lims = [-7.540, -7.534]

colours = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple','black', 'darkgrey']
labels = [f'r:58, a:28', f'r:140, a:68 (+0.2 m)', f'r:224, a: 108 (+0.4 m)', f'r:306, a:148 (+0.6 m)', f'r:388, a: 188 (+0.8 m)', f'MKA 1-5 (+2 m)', f'MKA 1-3 (+2.2 m)']
offsets = [0, 0.2, 0.4, 0.6, 0.8, 2, 2.2]


fig = plt.figure()

gs = GridSpec(4, 4, figure=fig)
ax00 = fig.add_subplot(gs[0, 0]) # win 1
ax01 = fig.add_subplot(gs[0, 1]) # win 2
ax02 = fig.add_subplot(gs[0, 2]) # win 3
ax10 = fig.add_subplot(gs[1, 0]) # win 4
ax11 = fig.add_subplot(gs[1, 1]) # win 5

ax03 = fig.add_subplot(gs[0, 3]) # mka 1-5
ax13 = fig.add_subplot(gs[1, 3]) # mka 1-3
ax2  = fig.add_subplot(gs[2:, :]) # swath profile


for (ax, obj,colour) in zip([ax00,ax01,ax02,ax10,ax11],example_pairs[0].Stack,colours):
    plot_data = ax.hexbin(obj.Lon_off.flatten(),obj.Lat_off.flatten(),C=obj.R_off.flatten(),gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
    ax.set_xlim(lon_lims)
    ax.set_ylim(lat_lims)
    ax.set_aspect('equal', 'box')
    ax.add_artist(ScaleBar(110123.8348,location='lower left'))
    # axes[0].set_axis_off()
    # cbar = plt.colorbar(plot_data,ax=ax)
    # cbar.set_label('Range offset [m]')
    ax.plot(np.asarray(buffer_poly.exterior.coords)[:,0],np.asarray(buffer_poly.exterior.coords)[:,1],color='black')
    ax.plot(np.asarray(line_coords)[:,0],np.asarray(line_coords[:,1]),color='red')
    # ax.set_axis_off()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
    ax.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:pink',label='L1992')
    ax.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
    ax.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
    ax.spines['bottom'].set_color(colour)
    ax.spines['top'].set_color(colour)
    ax.spines['left'].set_color(colour)
    ax.spines['right'].set_color(colour)
    [ax.spines[i].set_linewidth(2) for i in ax.spines]

ax03.hexbin(MKA_data_list[0].Lon_off_MKA_vec,MKA_data_list[0].Lat_off_MKA_vec,C=MKA_data_list[0].MKA_R_off_vec,gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax03.set_xlim(lon_lims)
ax03.set_ylim(lat_lims)
ax03.set_aspect('equal', 'box')
ax03.add_artist(ScaleBar(110123.8348,location='lower left'))

# cbar = plt.colorbar(plot_data,ax=ax03)
# cbar.set_label('Range offset [m]')
ax03.plot(np.asarray(buffer_poly.exterior.coords)[:,0],np.asarray(buffer_poly.exterior.coords)[:,1],color='black')
ax03.plot(np.asarray(line_coords)[:,0],np.asarray(line_coords[:,1]),color='red')
ax03.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
ax03.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:pink',label='L1992')
ax03.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
ax03.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
ax03.spines['bottom'].set_color('black')
ax03.spines['top'].set_color('black')
ax03.spines['left'].set_color('black')
ax03.spines['right'].set_color('black')
[ax03.spines[i].set_linewidth(2) for i in ax03.spines]
ax03.get_xaxis().set_visible(False)
ax03.get_yaxis().set_visible(False)

ax13.hexbin(MKA_data_list[1].Lon_off_MKA_vec,MKA_data_list[1].Lat_off_MKA_vec,C=MKA_data_list[1].MKA_R_off_vec,gridsize=grid_size,cmap=cmap,vmin=clims[0],vmax=clims[1])
ax13.set_xlim(lon_lims)
ax13.set_ylim(lat_lims)
ax13.set_aspect('equal', 'box')
ax13.add_artist(ScaleBar(110123.8348,location='lower left'))
# add_right_cax(ax13,0.05,0.02)
# cbar = plt.colorbar(plot_data,ax=ax13)
# cbar.set_label('Range offset [m]')
ax13.plot(np.asarray(buffer_poly.exterior.coords)[:,0],np.asarray(buffer_poly.exterior.coords)[:,1],color='black')
ax13.plot(np.asarray(line_coords)[:,0],np.asarray(line_coords[:,1]),color='red')

ax13.plot(coords_L1888[:,0],coords_L1888[:,1],linewidth=2,color='tab:blue',label='L1888')
ax13.plot(coords_L1992[:,0],coords_L1992[:,1],linewidth=2,color='tab:pink',label='L1992')
ax13.plot(coords_L1998[:,0],coords_L1998[:,1],linewidth=2,color='tab:green',label='L1998')
ax13.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
ax13.spines['bottom'].set_color('darkgrey')
ax13.spines['top'].set_color('darkgrey')
ax13.spines['left'].set_color('darkgrey')
ax13.spines['right'].set_color('darkgrey')
[ax13.spines[i].set_linewidth(2) for i in ax13.spines]
ax13.get_xaxis().set_visible(False)
ax13.get_yaxis().set_visible(False)


for i,(proj_point,colour,label, offset) in enumerate(zip(proj_point_list,colours,labels,offsets)):
    max_dist = proj_point_list[i].max()
    print(max_dist)
    edges = np.arange(0,max_dist,dx)
    profile_stats = np.empty(shape=(edges.shape[0]-1, 3), dtype=float) # pre-alloc nd-array
    for bin_id in enumerate(edges[:-1]):
        sel = np.squeeze(np.asarray([(proj_point[:,2]>=edges[bin_id[0]]) & (proj_point[:,2]<=edges[bin_id[0]+1])]))
        profile_stats[bin_id[0],:] = [np.nanpercentile(proj_point[sel,3],2.5),
                                np.nanmean(proj_point[sel,3]),
                                np.nanpercentile(proj_point[sel,3],97.5)]


    print(profile_stats)


    ax2.fill_between(edges[:-1]+dx/2 , profile_stats[:,0]+offset,profile_stats[:,2]+offset,alpha=0.3,color=colour)
    ax2.scatter(edges[:-1]+dx/2, profile_stats[:,1]+offset,marker='.',color=colour,label=label)
    ax2.set_xlabel('Distance along profile [m]')
    ax2.set_ylabel('Range offset [m]')
    # ax0.scatter(proj_point[:,2],proj_point[:,3],marker='.')
    ax2.legend(borderpad=0.1,labelspacing=0.2,ncol=2,frameon=False)
    ax2.set_ylim([-0.1,4.5])
    # plt.show()        

In [ ]:
plt.close('all')